<a href="https://colab.research.google.com/github/monacofj/sourcetide/blob/main/sourcetide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- **TideStats - Perofrmance metrics for collaborative development dynamics** -->

# SourceTide - Analytics for collaborative development dynamics
---










# USER SETUP

## Repository

In [1]:
# cellname : which-repo

# Repository identification

owner = "fossguild"  # <--- Replace with the actual owner username
repository_name = "naja"  # <--- Replace with the actual repository name

# GitHub Token
#
# To access the GitHub API, you'll need a personal access token. You can
# generate one in your GitHub account settings under "Developer settings" >
# "Personal  access tokens". Grant the token the necessary permissions
# (e.g., `repo` scope for full control or more specific scopes like
# `public_repo` for public repositories). In Colab, you can store your token
# securely in the secrets manager under the key icon in the left panel.
# Add a new secret with the name `GITHUB_TOKEN` and paste your token as
# the value.


## Period

In [2]:
# cellname : period

# Choose the period of analysis in weeks.

num_weeks = 1

## Report

In [3]:
# cellname : report

# Should an HTML report file be generated?
# Answer with 'y', 'Y' (for Yes) or 'n', 'N' (for No)

generate_report_value = 'y'

# Base path where the report folder will be saved (Google Drive)
report_path = 'My Drive/'


In [4]:
try:
    repo = g.get_user(owner).get_repo(repository_name)
    print(f"Repository Name: {repo.name}")
    print(f"Repository Description: {repo.description}")
    print(f"Repository URL: {repo.html_url}")
    print(f"Stars: {repo.stargazers_count}")
    print(f"Forks: {repo.forks_count}")
    print(f"Created At: {repo.created_at}")
    print(f"Last Updated At: {repo.updated_at}")

    # The analysis_run_date is now defined in the 'Analysis Run Date' cell in USER SETUP.
    # This cell will only print it.
    print(f"Analysis Run Date: {analysis_run_date}")

except Exception as e:
    print(f"Error fetching repository information: {e}")
    # If you see an error here AFTER replacing the owner and repository_name,
    # please copy and paste the full error message so I can assist you.


Error fetching repository information: name 'g' is not defined


## Análise

In [5]:
# cellname : analysis-date
from datetime import datetime

# Define the single source of truth for the analysis run date
analysis_run_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(f"Analysis started at: {analysis_run_date}")


Analysis started at: 2025-11-26 22:47:07


## Score

In [6]:
# cellname : score-parameters

## Chosse the parameters to be used to compute the contribution score
##
## activity = A * (REMI>0)   +   (1-A) (RDMI>0)
## reward   = R *  REMI      +   (1-R)  RDMI
## score    = S * activity   +   (1-S)  reward
##
## Interpretation:
##
## * activity a binary weighted measure of minimal participation in both
##            discussions and coding. The parameter A says how much each
##            dimension is important: A=1 means only discussion counts;
##            A=0 means only coding counts; A=0.5 both count equally.
##            Activity varies from 0 to 1.
##
##  * reward  is a weighted measure of participation in both discussion
##            and coding. The parameter R says how much each dimension is
##            important: R=0 means only discussion counts; R=1 means only
##            coding counts; R=0.5 both count equally.
##            Reward valies from 0 to 1.
##
##  * score   is a weighted combination of activity and reward. The parameter
##            S says which is more imporant: S=1 means that we're exclusively
##            interested in the fact that there was any participation, and
##            not in how intensive this participation was. S=0 means that the
##            score is proportional to the participation intensity. S=0.5
##            mean that half of the score is based on whether was any
##            participation, and the other half is a bonification for the
##            intensity of the work done.

A_factor = 0.5
R_factor = 0.5
S_factor = 0.5


# CONFIGURE TOOLS

In [7]:
from google.colab import drive
import os
import ipywidgets as widgets # Import ipywidgets to access the selector
from IPython.display import display, Markdown
from datetime import datetime

# Mount Google Drive
drive.mount('/content/gdrive')

# Define the report_folder_name using repository_name and analysis_run_date
# repository_name is from the 'which-repo' cell (borUqKOx6vVf)
# analysis_run_date is from the 'Get repository info' cell (-zplV57Ea4LY)

# Ensure analysis_run_date is available (defined in -zplV57Ea4LY)
if 'analysis_run_date' not in globals():
    # Fallback if analysis_run_date hasn't been set yet
    current_date_str = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
else:
    # Use the date from the analysis, formatted for folder names
    # analysis_run_date is already a string '%Y-%m-%d %H:%M:%S', reformat for folder name
    current_date_obj = datetime.strptime(analysis_run_date, '%Y-%m-%d %H:%M:%S')
    current_date_str = current_date_obj.strftime('%Y-%m-%d_%H-%M-%S')

report_folder_name = f"{repository_name}-Tide-{current_date_str}"

# Define the output folder path using report_path (from 'report' cell) and report_folder_name
# Ensure report_path is defined in the 'report' cell (OnyMCQQSVi00)
output_folder = os.path.join('/content/gdrive', report_path, report_folder_name)

# Create the folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)
print(f"Output folder '{output_folder}' ensured to exist.")


Mounted at /content/gdrive
Output folder '/content/gdrive/My Drive/naja-2025-11-26_22-47-07' ensured to exist.


In [8]:
# cellname : prepare-tools

!pip --quiet install PyGithub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.9 MB/s eta 0:00:00


In [9]:
import plotly.io as pio
import os
from IPython.display import display, Markdown

def save_plotly_figure_as_html(fig, filename):
    """
    Saves a Plotly figure as an interactive HTML file in the designated output folder.

    Args:
        fig (plotly.graph_objects.Figure): The Plotly figure object to save.
        filename (str): The desired filename (e.g., 'my_plot.html').
    """
    global output_folder

    # Ensure filename has .html extension
    if not filename.lower().endswith('.html'):
        filename = os.path.splitext(filename)[0] + '.html'

    full_path = os.path.join(output_folder, filename)
    pio.write_html(fig, full_path, auto_open=False) # auto_open=False to prevent browser from opening
    print(f"Figure saved to: {full_path}")


def save_fig(fig, filename):
    """
    Conditionally saves a Plotly figure as an interactive HTML file
    if report generation is enabled via the 'generate_report_value' variable.

    Args:
        fig (plotly.graph_objects.Figure): The Plotly figure object to save.
        filename (str): The desired filename (e.g., 'my_plot.html').
    """
    # Access the global variable 'generate_report_value' directly
    if 'generate_report_value' in globals() and generate_report_value.lower() == 'y':
        save_plotly_figure_as_html(fig, filename)
    else:
        print(f"Report generation is not enabled. Skipping saving '{filename}'.")

print("Function 'save_fig' defined.")

Function 'save_fig' defined.


In [10]:
from datetime import timedelta
from IPython.display import display, Markdown

# Calculate the corresponding timedelta from num_weeks (defined in the 'period' cell)
selected_timedelta = timedelta(weeks=num_weeks)

# Create a string representation for the report
selected_period_str = f'{num_weeks} weeks'

# Display the selected period for confirmation
display(Markdown(f'**Analysis Period: {selected_period_str}**'))

**Analysis Period: 1 weeks**

In [11]:
# cellname : token
import os
from google.colab import userdata
from github import Github

# Retrieve the GitHub token from Colab secrets
github_token = userdata.get('GITHUB_TOKEN')

if github_token is None:
    print("GitHub token not found. Please add it to Colab secrets with the name 'GITHUB_TOKEN'.")
else:
    # Authenticate with the GitHub API
    g = Github(github_token)
    print("Successfully authenticated with GitHub API.")

Successfully authenticated with GitHub API.


/tmp/ipython-input-1902249742.py:13: DeprecationWarning: Argument login_or_token is deprecated, please use auth=github.Auth.Token(...) instead
  g = Github(github_token)


# FETCH DEVELOPERS

## Get repository info

In [12]:
try:
    repo = g.get_user(owner).get_repo(repository_name)
    print(f"Repository Name: {repo.name}")
    print(f"Repository Description: {repo.description}")
    print(f"Repository URL: {repo.html_url}")
    print(f"Stars: {repo.stargazers_count}")
    print(f"Forks: {repo.forks_count}")
    print(f"Created At: {repo.created_at}")
    print(f"Last Updated At: {repo.updated_at}")

    # Capture and print the current analysis run date
    from datetime import datetime # Ensure datetime is imported
    analysis_run_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f"Analysis Run Date: {analysis_run_date}")

except Exception as e:
    print(f"Error fetching repository information: {e}")
    # If you see an error here AFTER replacing the owner and repository_name,
    # please copy and paste the full error message so I can assist you.

Repository Name: naja
Repository Description: Naja is an implementation of the Snake Game based on KobraPy programming exercise (2025)
Repository URL: https://github.com/fossguild/naja
Stars: 17
Forks: 39
Created At: 2025-10-01 18:48:04+00:00
Last Updated At: 2025-11-26 21:09:44+00:00
Analysis Run Date: 2025-11-26 22:47:36


## Get contributors
All people that contributed to the project, even if not with repository access (i.e. including external contributors).

In [13]:
raw_contributors = repo.get_contributors() # Fetch all contributors first

filtered_contributors = []
bot_logins_to_exclude = ['Copilot'] # Explicitly exclude Copilot as it caused 404 in previous cells

for contributor in raw_contributors:
    # Check if contributor is a bot based on type or common naming conventions
    is_bot = (
        contributor.type == 'Bot' or
        contributor.login.endswith('[bot]') or
        contributor.login in bot_logins_to_exclude
    )

    if not is_bot:
        filtered_contributors.append(contributor)

all_contributors = filtered_contributors # Update all_contributors with the filtered list

print(f"Found {len(all_contributors)} non-bot contributors.")

# Display the login, name, and number of contributions for the first few non-bot contributors
print("\nFirst 5 non-bot contributors:")
for i, contributor in enumerate(all_contributors):
    if i >= 5:
        break
    # Get contributor name, handling cases where it might be None
    contributor_name = contributor.name if contributor.name else "N/A"
    # Use f-string formatting to align output
    print(f"- {contributor.login:<20}  {contributor_name:<30}    Contributions: {contributor.contributions}")

Found 38 non-bot contributors.

First 5 non-bot contributors:
- lfelipediniz          Luiz Felipe                       Contributions: 42
- LeticiaBN             Leticia Barbosa Neves             Contributions: 15
- caue-paiva            Caue Paiva                        Contributions: 9
- monacofj              Monaco F. J.                      Contributions: 7
- bdogabriel            Gabriel Barbosa                   Contributions: 6


## Get collaborators
This is a list of everyone with access to the repostory, even those who never effectively contributed.

In [14]:
collaborators = repo.get_collaborators()

print(f"Found {collaborators.totalCount} collaborators with access to the repository.")

print("\nFirst 5 collaborators:")
for i, collaborator in enumerate(collaborators):
    if i >= 5:
        break
    # Get collaborator name, handling cases where it might be None
    collaborator_name = collaborator.name if collaborator.name else "N/A"

    permissions_list = []
    if collaborator.permissions.admin: permissions_list.append('admin')
    if collaborator.permissions.push: permissions_list.append('push')
    if collaborator.permissions.pull: permissions_list.append('pull')

    print(f"- {collaborator.login:<20}  Name: {collaborator_name:<30}  Permissions: {', '.join(permissions_list) if permissions_list else 'None'}")

Found 65 collaborators with access to the repository.

First 5 collaborators:
- monacofj              Name: Monaco F. J.                    Permissions: admin, push, pull
- LiviaLelis            Name: Lívia Lelis                     Permissions: push, pull
- trqt                  Name: Guilherme Torquato              Permissions: push, pull
- jotaefepinho          Name: João Francisco Pinho            Permissions: push, pull
- chonkachu             Name: N/A                             Permissions: push, pull


## Consolidate developers
Create a dataframe with all collaborators (people with repository access), plus anyone else that has contributed (external contributors).

In [15]:
import pandas as pd

# --- 1. Prepare base information for all unique individuals (collaborators + external contributors) ---

# Dictionary to store basic details for each unique person
# Key: login (str), Value: {'name': str, 'access': str}
person_base_info = {}

# First, add all collaborators with 'yes' access status
# This also populates their names
collaborator_logins = set()
for coll in collaborators:
    collaborator_logins.add(coll.login)
    person_base_info[coll.login] = {
        'name': coll.name if coll.name else 'N/A',
        'access': 'yes'
    }

# Now, add all_contributors who are NOT collaborators with 'no' access status
for contr in all_contributors:
    if contr.login not in person_base_info:
        # This person is a contributor but not a direct collaborator
        person_base_info[contr.login] = {
            'name': contr.name if contr.name else 'N/A',
            'access': 'no'
        }
    else:
        # If a person is both a contributor and a collaborator, ensure their name is captured
        # in case the contributor object had a better name than the collaborator object
        if person_base_info[contr.login]['name'] == 'N/A' and contr.name:
            person_base_info[contr.login]['name'] = contr.name

# Convert this base info to a DataFrame
df_people = pd.DataFrame.from_dict(person_base_info, orient='index')
df_people.index.name = 'github_login'
df_people = df_people.reset_index()

print("Unified DataFrame of Collaborators and External Contributors:")
display(df_people.head())

#print("\nDataFrame Info:")
#df_people.info()

Unified DataFrame of Collaborators and External Contributors:


,github_login,name,access
0,monacofj,Monaco F. J.,yes
1,LiviaLelis,Lívia Lelis,yes
2,trqt,Guilherme Torquato,yes
3,jotaefepinho,João Francisco Pinho,yes
4,chonkachu,N/A,yes


# FETCH EVENTS

In [16]:
from datetime import timedelta

# selected_timedelta is now calculated directly in the 'period' cell.
# This cell now just confirms the value.

print(f"Corresponding timedelta for analysis: {selected_timedelta}")

Corresponding timedelta for analysis: 7 days, 0:00:00


In [17]:
num_core_devs = df_people[df_people['access'] == 'yes'].shape[0]
num_externals = df_people[df_people['access'] == 'no'].shape[0]

print(f"* Number of collaborators (core devs): {num_core_devs}")
print(f"* Number of contributors without repo access (externals): {num_externals}")

* Number of collaborators (core devs): 65
* Number of contributors without repo access (externals): 2


## Issues



In [18]:

## Get issues open

import os
from google.colab import userdata
from github import Github
from datetime import datetime, timedelta, timezone
import pandas as pd

# Calculate the start date based on the selected_timedelta
# Ensure selected_timedelta is available from previous cells
start_date = datetime.now(timezone.utc) - selected_timedelta

# Dictionary to store issues opened count per user
issues_opened_count = {}

# Fetch all issues since the start_date. This fetches issues CREATED or UPDATED since start_date.
issues = repo.get_issues(state='all', since=start_date)

for issue in issues:
    # Only count issues that were CREATED within the selected period
    if issue.user and issue.created_at >= start_date:
        login = issue.user.login
        issues_opened_count[login] = issues_opened_count.get(login, 0) + 1

# Convert the dictionary to a pandas Series for merging
issues_opened_series = pd.Series(issues_opened_count, name='issues_opened_count')

# --- Fix for KeyError: Drop existing 'issues_opened_count' related columns before merging ---
columns_to_drop = [col for col in df_people.columns if col.startswith('issues_opened_count')]
if columns_to_drop:
    df_people = df_people.drop(columns=columns_to_drop)

df_people = df_people.set_index('github_login')
df_people = df_people.merge(issues_opened_series, left_index=True, right_index=True, how='left')
df_people = df_people.reset_index()

# Fill NaN values with 0 for users who didn't open any issues in the period
df_people['issues_opened_count'] = df_people['issues_opened_count'].fillna(0).astype(int)

print("df_people DataFrame with 'issues_opened_count' column:")
display(df_people.head())

print("\nTop 4 people with most issues opened:")
top_issues_openers = df_people.sort_values(by='issues_opened_count', ascending=False).head(4)
for index, row in top_issues_openers.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['issues_opened_count']} issues opened")

df_people DataFrame with 'issues_opened_count' column:


,github_login,name,access,issues_opened_count
0,monacofj,Monaco F. J.,yes,1
1,LiviaLelis,Lívia Lelis,yes,0
2,trqt,Guilherme Torquato,yes,2
3,jotaefepinho,João Francisco Pinho,yes,0
4,chonkachu,N/A,yes,0



Top 4 people with most issues opened:
- Augusto-Ildefonso (Augusto Fernandes Ildefonso): 4 issues opened
- Nikk0o (Niko): 3 issues opened
- JuanHPassos (Juan Henriques Passos): 3 issues opened
- DeguShi (Felipe Azambuja): 3 issues opened


In [19]:

## Get issues reopened

from datetime import datetime, timezone
import pandas as pd

# Ensure selected_timedelta is available from previous cells
start_date = datetime.now(timezone.utc) - selected_timedelta

# Dictionary to store issues reopened count per user
issues_reopened_count = {}

# Fetch all issues that were created or updated since the start_date
# This gives us a pool of issues that might have been reopened in the period
issues_in_period = repo.get_issues(state='all', since=start_date)

for issue in issues_in_period:
    try:
        # Get events for this specific issue (timeline events)
        issue_timeline_events = issue.get_events()
        for event in issue_timeline_events:
            # Check if the event is a 'reopened' event, happened in the period, and has an actor
            if event.event == 'reopened' and event.created_at >= start_date and event.actor:
                login = event.actor.login
                issues_reopened_count[login] = issues_reopened_count.get(login, 0) + 1
    except Exception as e:
        # Handle potential errors if issue events cannot be fetched (e.g., rate limits, deleted events)
        print(f"Warning: Could not fetch events for issue #{issue.number}: {e}")

# Convert the dictionary to a pandas Series for merging
issues_reopened_series = pd.Series(issues_reopened_count, name='issues_reopened_count')

# Merge the counts into df_people
df_people = df_people.set_index('github_login')
df_people = df_people.merge(issues_reopened_series, left_index=True, right_index=True, how='left')
df_people = df_people.reset_index()

# Fill NaN values with 0 for users who didn't reopen any issues in the period
df_people['issues_reopened_count'] = df_people['issues_reopened_count'].fillna(0).astype(int)

print("df_people DataFrame with 'issues_reopened_count' column:")
display(df_people.head())

print("\nTop 4 people with most issues reopened:")
top_reopeners = df_people.sort_values(by='issues_reopened_count', ascending=False).head(4)
for index, row in top_reopeners.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['issues_reopened_count']} issues reopened")

df_people DataFrame with 'issues_reopened_count' column:


,github_login,name,access,issues_opened_count,issues_reopened_count
0,monacofj,Monaco F. J.,yes,1,0
1,LiviaLelis,Lívia Lelis,yes,0,0
2,trqt,Guilherme Torquato,yes,2,0
3,jotaefepinho,João Francisco Pinho,yes,0,0
4,chonkachu,N/A,yes,0,0



Top 4 people with most issues reopened:
- Augusto-Ildefonso (Augusto Fernandes Ildefonso): 1 issues reopened
- LiviaLelis (Lívia Lelis): 0 issues reopened
- monacofj (Monaco F. J.): 0 issues reopened
- trqt (Guilherme Torquato): 0 issues reopened


## Comments

PR are treated as issues with code.

In [20]:
from datetime import datetime, timezone
import pandas as pd

# Ensure selected_timedelta is available from previous cells
start_date = datetime.now(timezone.utc) - selected_timedelta

# Dictionary to store IssueCommentEvent counts per user
issue_comment_event_counts = {}

# --- Process Issue Comments ---
# Fetch all issues since the start_date (this method supports 'since')
issues_in_period = repo.get_issues(state='all', since=start_date)

for issue in issues_in_period:
    # Fetch comments for each issue
    comments = issue.get_comments()
    for comment in comments:
        # Ensure the comment itself was created within the period and has a user
        if comment.user and comment.created_at >= start_date:
            login = comment.user.login
            issue_comment_event_counts[login] = issue_comment_event_counts.get(login, 0) + 1

# --- Process Pull Request Comments (which are also IssueCommentEvent conceptually) ---
# Fetch all pull requests (this method does NOT support 'since')
pulls_in_period = repo.get_pulls(state='all')

for pull in pulls_in_period:
    # Fetch comments for each pull request
    comments = pull.get_comments()
    for comment in comments:
        # Ensure the comment itself was created within the period and has a user
        if comment.user and comment.created_at >= start_date:
            login = comment.user.login
            issue_comment_event_counts[login] = issue_comment_event_counts.get(login, 0) + 1

# Convert the dictionary to a pandas Series for merging
issue_comment_event_series = pd.Series(issue_comment_event_counts, name='issue_comment_event_count')

# Merge the counts into df_people
df_people = df_people.set_index('github_login')
df_people = df_people.merge(issue_comment_event_series, left_index=True, right_index=True, how='left')
df_people = df_people.reset_index()

# Fill NaN values with 0 for users who didn't perform any IssueCommentEvent in the period
df_people['issue_comment_event_count'] = df_people['issue_comment_event_count'].fillna(0).astype(int)

print("df_people DataFrame with 'issue_comment_event_count' column:")
display(df_people.head())

print("\nTop 4 people with most IssueCommentEvents:")
top_commenters = df_people.sort_values(by='issue_comment_event_count', ascending=False).head(4)
for index, row in top_commenters.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['issue_comment_event_count']} IssueCommentEvents")

df_people DataFrame with 'issue_comment_event_count' column:


,github_login,name,access,issues_opened_count,issues_reopened_count,issue_comment_event_count
0,monacofj,Monaco F. J.,yes,1,0,0
1,LiviaLelis,Lívia Lelis,yes,0,0,0
2,trqt,Guilherme Torquato,yes,2,0,4
3,jotaefepinho,João Francisco Pinho,yes,0,0,2
4,chonkachu,N/A,yes,0,0,0



Top 4 people with most IssueCommentEvents:
- davigdomingues (Davi Gabriel Domingues): 9 IssueCommentEvents
- DeguShi (Felipe Azambuja): 6 IssueCommentEvents
- otaviobia (Otávio Biagioni): 5 IssueCommentEvents
- gusvidigal (Name not available): 4 IssueCommentEvents


## Pull requests

In [21]:
from datetime import datetime, timezone
import pandas as pd

# Ensure selected_timedelta is available from previous cells
start_date = datetime.now(timezone.utc) - selected_timedelta

# Dictionary to store pull requests opened count per user
pull_requests_opened_count = {}
# Dictionary to store pull requests merged count per user
pull_requests_merged_count = {}
# Dictionary to store pull requests closed (including merged) count per user
pull_requests_closed_count = {}

# Fetch all pull requests (this method does NOT support 'since' for 'all' state directly, but can filter later)
# It's generally better to iterate through all and filter by date.
all_pulls = repo.get_pulls(state='all')

for pull in all_pulls:
    # Only count pull requests that were CREATED within the selected period
    if pull.user and pull.created_at >= start_date:
        login = pull.user.login
        pull_requests_opened_count[login] = pull_requests_opened_count.get(login, 0) + 1

    # Count pull requests that were MERGED within the selected period
    if pull.merged and pull.merged_at and pull.merged_at >= start_date:
        if pull.merged_by:
            login = pull.merged_by.login
            pull_requests_merged_count[login] = pull_requests_merged_count.get(login, 0) + 1

    # Count pull requests that were CLOSED (either merged or closed without merging) within the selected period
    # Note: 'closed_at' is typically populated for both merged and explicitly closed PRs.
    # We attribute the closure to the user who opened the PR for PI calculation.
    if pull.user and pull.state == 'closed' and pull.closed_at and pull.closed_at >= start_date:
        login = pull.user.login # User who created the PR
        pull_requests_closed_count[login] = pull_requests_closed_count.get(login, 0) + 1

# Convert the dictionaries to pandas Series for merging
pull_requests_opened_series = pd.Series(pull_requests_opened_count, name='pull_requests_opened_count')
pull_requests_merged_series = pd.Series(pull_requests_merged_count, name='pull_requests_merged_count')
pull_requests_closed_series = pd.Series(pull_requests_closed_count, name='pull_requests_closed_count')


# --- Drop existing 'pull_requests_opened_count', 'pull_requests_merged_count', 'pull_requests_closed_count' related columns before merging ---
columns_to_drop_opened = [col for col in df_people.columns if col.startswith('pull_requests_opened_count')]
if columns_to_drop_opened:
    df_people = df_people.drop(columns=columns_to_drop_opened)

columns_to_drop_merged = [col for col in df_people.columns if col.startswith('pull_requests_merged_count')]
if columns_to_drop_merged:
    df_people = df_people.drop(columns=columns_to_drop_merged)

columns_to_drop_closed = [col for col in df_people.columns if col.startswith('pull_requests_closed_count')]
if columns_to_drop_closed:
    df_people = df_people.drop(columns=columns_to_drop_closed)


df_people = df_people.set_index('github_login')

# Merge counts
df_people = df_people.merge(pull_requests_opened_series, left_index=True, right_index=True, how='left')
df_people = df_people.merge(pull_requests_merged_series, left_index=True, right_index=True, how='left')
df_people = df_people.merge(pull_requests_closed_series, left_index=True, right_index=True, how='left')

df_people = df_people.reset_index()

# Fill NaN values with 0
df_people['pull_requests_opened_count'] = df_people['pull_requests_opened_count'].fillna(0).astype(int)
df_people['pull_requests_merged_count'] = df_people['pull_requests_merged_count'].fillna(0).astype(int)
df_people['pull_requests_closed_count'] = df_people['pull_requests_closed_count'].fillna(0).astype(int)

print("df_people DataFrame with PR counts:")
display(df_people.head())

print("\nTop 4 people with most Pull Requests opened:")
top_pr_openers = df_people.sort_values(by='pull_requests_opened_count', ascending=False).head(4)
for index, row in top_pr_openers.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['pull_requests_opened_count']} PRs opened")

print("\nTop 4 people with most Pull Requests merged:")
top_pr_mergers = df_people.sort_values(by='pull_requests_merged_count', ascending=False).head(4)
for index, row in top_pr_mergers.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['pull_requests_merged_count']} PRs merged")

print("\nTop 4 people with most Pull Requests closed:")
top_pr_closed = df_people.sort_values(by='pull_requests_closed_count', ascending=False).head(4)
for index, row in top_pr_closed.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['pull_requests_closed_count']} PRs closed")

df_people DataFrame with PR counts:


,github_login,name,access,issues_opened_count,issues_reopened_count,issue_comment_event_count,pull_requests_opened_count,pull_requests_merged_count,pull_requests_closed_count
0,monacofj,Monaco F. J.,yes,1,0,0,0,0,0
1,LiviaLelis,Lívia Lelis,yes,0,0,0,0,1,1
2,trqt,Guilherme Torquato,yes,2,0,4,1,1,1
3,jotaefepinho,João Francisco Pinho,yes,0,0,2,0,0,0
4,chonkachu,N/A,yes,0,0,0,0,0,0



Top 4 people with most Pull Requests opened:
- JuanHPassos (Juan Henriques Passos): 2 PRs opened
- DeguShi (Felipe Azambuja): 2 PRs opened
- trqt (Guilherme Torquato): 1 PRs opened
- LeticiaBN (Leticia Barbosa Neves): 1 PRs opened

Top 4 people with most Pull Requests merged:
- davigdomingues (Davi Gabriel Domingues): 2 PRs merged
- LeticiaBN (Leticia Barbosa Neves): 2 PRs merged
- otaviobia (Otávio Biagioni): 2 PRs merged
- DeguShi (Felipe Azambuja): 1 PRs merged

Top 4 people with most Pull Requests closed:
- JuanHPassos (Juan Henriques Passos): 2 PRs closed
- trqt (Guilherme Torquato): 1 PRs closed
- LiviaLelis (Lívia Lelis): 1 PRs closed
- Nikk0o (Niko): 1 PRs closed


## Code changes

In [22]:
from datetime import datetime, timezone
import pandas as pd

# Ensure selected_timedelta is available from previous cells
start_date = datetime.now(timezone.utc) - selected_timedelta

# Dictionaries to store additions and deletions per user
user_additions = {}
user_deletions = {}

print(f"Fetching code changes from merged and currently open Pull Requests since {start_date}...")

# Fetch all pull requests (open, closed, and all states to check merged status)
all_pulls = repo.get_pulls(state='all')

for pull in all_pulls:
    process_pull = False
    # Condition 1: PR was merged within the period
    if pull.merged and pull.merged_at and pull.merged_at >= start_date:
        process_pull = True
    # Condition 2: PR is currently open
    elif pull.state == 'open':
        process_pull = True

    if process_pull:
        # Iterate through the commits of the relevant PR
        try:
            pr_commits = pull.get_commits()
            for commit in pr_commits:
                # Check if the commit's author exists
                if commit.author and commit.author.login:
                    login = commit.author.login
                    try:
                        full_commit = repo.get_commit(commit.sha)
                        if full_commit.stats:
                            user_additions[login] = user_additions.get(login, 0) + full_commit.stats.additions
                            user_deletions[login] = user_deletions.get(login, 0) + full_commit.stats.deletions
                    except Exception as e:
                        print(f"Warning: Could not fetch stats for commit {commit.sha} in PR #{pull.number} by {login}: {e}")
        except Exception as e:
            print(f"Warning: Could not fetch commits for PR #{pull.number}: {e}")

# Convert dictionaries to pandas Series
additions_series = pd.Series(user_additions, name='total_additions')
deletions_series = pd.Series(user_deletions, name='total_deletions')

# --- Drop existing columns before merging ---
# This prevents duplicate columns if the cell is run multiple times
columns_to_drop_add = [col for col in df_people.columns if col.startswith('total_additions')]
if columns_to_drop_add:
    df_people = df_people.drop(columns=columns_to_drop_add)

columns_to_drop_del = [col for col in df_people.columns if col.startswith('total_deletions')]
if columns_to_drop_del:
    df_people = df_people.drop(columns=columns_to_drop_del)

df_people = df_people.set_index('github_login')

# Merge additions and deletions into df_people
df_people = df_people.merge(additions_series, left_index=True, right_index=True, how='left')
df_people = df_people.merge(deletions_series, left_index=True, right_index=True, how='left')

df_people = df_people.reset_index()

# Fill NaN values with 0 for users who had no additions or deletions in the period
df_people['total_additions'] = df_people['total_additions'].fillna(0).astype(int)
df_people['total_deletions'] = df_people['total_deletions'].fillna(0).astype(int)

print("\ndf_people DataFrame with 'total_additions' and 'total_deletions' columns:")
display(df_people.head())

print("\nTop 4 people with most additions:")
top_adders = df_people.sort_values(by='total_additions', ascending=False).head(4)
for index, row in top_adders.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['total_additions']} additions")

print("\nTop 4 people with most deletions:")
top_deleters = df_people.sort_values(by='total_deletions', ascending=False).head(4)
for index, row in top_deleters.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['total_deletions']} deletions")

Fetching code changes from merged and currently open Pull Requests since 2025-11-19 22:52:38.423320+00:00...

df_people DataFrame with 'total_additions' and 'total_deletions' columns:


,github_login,name,access,issues_opened_count,issues_reopened_count,issue_comment_event_count,pull_requests_opened_count,pull_requests_merged_count,pull_requests_closed_count,total_additions,total_deletions
0,monacofj,Monaco F. J.,yes,1,0,0,0,0,0,0,0
1,LiviaLelis,Lívia Lelis,yes,0,0,0,0,1,1,1832,250
2,trqt,Guilherme Torquato,yes,2,0,4,1,1,1,64,6
3,jotaefepinho,João Francisco Pinho,yes,0,0,2,0,0,0,0,0
4,chonkachu,N/A,yes,0,0,0,0,0,0,0,0



Top 4 people with most additions:
- MatheusPaivaa (Matheus Paiva): 3222 additions
- davigdomingues (Davi Gabriel Domingues): 2059 additions
- LiviaLelis (Lívia Lelis): 1832 additions
- pecazeco (Pedro Azevedo): 1269 additions

Top 4 people with most deletions:
- DeguShi (Felipe Azambuja): 424 deletions
- MatheusPaivaa (Matheus Paiva): 391 deletions
- davigdomingues (Davi Gabriel Domingues): 261 deletions
- LiviaLelis (Lívia Lelis): 250 deletions


In [23]:
# cellname : fix-count-scales

import pandas as pd
import numpy as np

# 2. Define a list named other_event_cols
other_event_cols = [
    'issues_opened_count',
    'issues_reopened_count',
    'issue_comment_event_count',
    'pull_requests_opened_count',
    'pull_requests_merged_count'
]

# Ensure all columns exist, filter out non-existent ones if any
existing_other_event_cols = [col for col in other_event_cols if col in df_people.columns]

# 3. Calculate max_other_event_count
if existing_other_event_cols:
    max_other_event_count = df_people[existing_other_event_cols].max().max()
else:
    max_other_event_count = 0

# Avoid division by zero if no other events
if max_other_event_count == 0:
    max_other_event_count = 1

# 4. Calculate max_total_additions and max_total_deletions
max_total_additions = df_people['total_additions'].max()
max_total_deletions = df_people['total_deletions'].max()

# Determine max_code_change as the maximum value between max_total_additions and max_total_deletions
max_code_change = max(max_total_additions, max_total_deletions)

# Calculate the scaling_ratio by dividing max_code_change by max_other_event_count
# Handle the case where max_other_event_count is zero to avoid division errors.
scaling_ratio = max_code_change / max_other_event_count if max_other_event_count > 0 else 0

# Based on scaling_ratio, determine the chosen_factor:
chosen_factor = 1 # Default value
if max_code_change == 0: # If there are no code changes, factor is 1
    chosen_factor = 1
elif scaling_ratio <= 1:
    chosen_factor = 1
elif 1 < scaling_ratio <= 10:
    chosen_factor = 10
elif scaling_ratio > 10 and scaling_ratio <= 100:
    chosen_factor = int(np.ceil(scaling_ratio / 10)) * 10
elif scaling_ratio > 100:
    chosen_factor = int(np.ceil(scaling_ratio / 100)) * 100

# Create a scaled_label_suffix string
if chosen_factor == 1:
    scaled_label_suffix = ''
else:
    scaled_label_suffix = f' (x{chosen_factor})'

# 5. Initialize new columns (or overwrite if they exist)
df_people['scaled_additions'] = 0.0
df_people['scaled_deletions'] = 0.0

# Update the scaled_additions column in df_people
df_people['scaled_additions'] = df_people['total_additions'] / chosen_factor

# Update the scaled_deletions column in df_people
df_people['scaled_deletions'] = df_people['total_deletions'] / chosen_factor

# 8. Print the calculated values
print(f"Maximum count of other event types (for scaling): {max_other_event_count}")
print(f"Maximum total additions: {max_total_additions}")
print(f"Maximum total deletions: {max_total_deletions}")
print(f"Maximum code change (additions or deletions): {max_code_change}")
if max_code_change > 0 and max_other_event_count > 0:
    print(f"Scaling ratio (max_code_change / max_other_event_count): {scaling_ratio:.4f}")
print(f"Chosen factor for scaling code changes: {chosen_factor}")
print(f"Scaled label suffix: '{scaled_label_suffix}'")

# 9. Display the head of the df_people DataFrame
print("\ndf_people DataFrame with scaled additions and deletions:")
display(df_people[['github_login', 'name', 'total_additions', 'scaled_additions', 'total_deletions', 'scaled_deletions']].head())

# 10. Print top 4 contributors by scaled_additions
print("\nTop 4 people by scaled additions:")
top_scaled_adders = df_people.sort_values(by='scaled_additions', ascending=False).head(4)
for index, row in top_scaled_adders.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['scaled_additions']:.2f} scaled additions")

# 11. Print top 4 contributors by scaled_deletions
print("\nTop 4 people by scaled deletions:")
top_scaled_deleters = df_people.sort_values(by='scaled_deletions', ascending=False).head(4)
for index, row in top_scaled_deleters.iterrows():
    print(f"- {row['github_login']} ({row['name'] if row['name'] != 'N/A' else 'Name not available'}): {row['scaled_deletions']:.2f} scaled deletions")

Maximum count of other event types (for scaling): 9
Maximum total additions: 3222
Maximum total deletions: 424
Maximum code change (additions or deletions): 3222
Scaling ratio (max_code_change / max_other_event_count): 358.0000
Chosen factor for scaling code changes: 400
Scaled label suffix: ' (x400)'

df_people DataFrame with scaled additions and deletions:


,github_login,name,total_additions,scaled_additions,total_deletions,scaled_deletions
0,monacofj,Monaco F. J.,0,0.00,0,0.000
1,LiviaLelis,Lívia Lelis,1832,4.58,250,0.625
2,trqt,Guilherme Torquato,64,0.16,6,0.015
3,jotaefepinho,João Francisco Pinho,0,0.00,0,0.000
4,chonkachu,N/A,0,0.00,0,0.000



Top 4 people by scaled additions:
- MatheusPaivaa (Matheus Paiva): 8.05 scaled additions
- davigdomingues (Davi Gabriel Domingues): 5.15 scaled additions
- LiviaLelis (Lívia Lelis): 4.58 scaled additions
- pecazeco (Pedro Azevedo): 3.17 scaled additions

Top 4 people by scaled deletions:
- DeguShi (Felipe Azambuja): 1.06 scaled deletions
- MatheusPaivaa (Matheus Paiva): 0.98 scaled deletions
- davigdomingues (Davi Gabriel Domingues): 0.65 scaled deletions
- LiviaLelis (Lívia Lelis): 0.62 scaled deletions


# PROJECT OVERVIEW

## Primitive events

* Show all events in the period (ommit devs that haven't generated events)

In [24]:
# cellname : show-all-events

import plotly.express as px
import pandas as pd

# Diagnostic: Print all columns in df_people before processing for the plot
print("Current columns in df_people before plotting:")
print(df_people.columns)

# Define the columns that represent events, NOW USING SCALED ADDITIONS AND DELETIONS
event_columns = [
    'issues_opened_count',
    'issues_reopened_count',
    'issue_comment_event_count',
    'pull_requests_opened_count',
    'pull_requests_merged_count',
    'scaled_additions',  # Use scaled additions
    'scaled_deletions'   # Use scaled deletions
]

# Ensure all event columns exist in df_people
existing_event_columns = [col for col in event_columns if col in df_people.columns]

# Filter out developers with no activity in any of the tracked events for cleaner visualization
df_plot = df_people[df_people[existing_event_columns].sum(axis=1) > 0].copy()

# Rename columns for better plot labels (optional, but good practice)
df_plot = df_plot.rename(columns={
    'issues_opened_count': 'Issues Opened',
    'issues_reopened_count': 'Issues Reopened',
    'issue_comment_event_count': 'Comments (Issues/PRs)',
    'pull_requests_opened_count': 'PRs Opened',
    'pull_requests_merged_count': 'PRs Merged',
    'scaled_additions': f'Lines Added{scaled_label_suffix}', # Updated label with suffix
    'scaled_deletions': f'Lines Removed{scaled_label_suffix}' # Updated label with suffix
})

# Melt the DataFrame to a long format suitable for stacked bar charts
# We use 'github_login' as the id_vars because we want one bar per developer
df_melted = df_plot.melt(id_vars=['github_login', 'name'], value_vars=[col for col in df_plot.columns if col in [f'Lines Added{scaled_label_suffix}', f'Lines Removed{scaled_label_suffix}', 'Issues Opened', 'Issues Reopened', 'Comments (Issues/PRs)', 'PRs Opened', 'PRs Merged']],
                         var_name='Event Type', value_name='Count')

# Create the stacked bar chart using Plotly Express
fig = px.bar(df_melted,
             x='github_login',
             y='Count',
             color='Event Type',
             title=f'Event count per Contributor', # Updated title with suffix
             labels={'github_login': 'Contributor', 'Count': 'Event Count'},
             hover_name='name',
             hover_data={'Event Type': True, 'Count': ':.2f', 'github_login': False}) # Format Count to 2 decimal places

fig.update_layout(xaxis_title='Contributor',
                  yaxis_title='Event Count',
                  barmode='stack',
                  legend_title='Event Type')

# Optional: Adjust x-axis to show only relevant developer names if too many
# fig.update_xaxes(tickangle=45, tickfont=dict(size=10))

fig.show() # Always show the figure

# Save the figure to Google Drive if report generation is enabled
save_fig(fig, 'all_primitive_events.html')

Current columns in df_people before plotting:
Index(['github_login', 'name', 'access', 'issues_opened_count',
       'issues_reopened_count', 'issue_comment_event_count',
       'pull_requests_opened_count', 'pull_requests_merged_count',
       'pull_requests_closed_count', 'total_additions', 'total_deletions',
       'scaled_additions', 'scaled_deletions'],
      dtype='object')


Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/all_primitive_events.html


## Agregate events
* All events, but aggregating Issues, Commands,PRs and total changes

In [25]:
# cellname : all-events-aggregate
import plotly.express as px
import pandas as pd

# Create consolidated event columns
df_consolidated = df_people.copy()
df_consolidated['Issues'] = df_consolidated['issues_opened_count'] + df_consolidated['issues_reopened_count']
df_consolidated['Comments'] = df_consolidated['issue_comment_event_count']
df_consolidated['PRs Opened'] = df_consolidated['pull_requests_opened_count']
df_consolidated['PRs Merged'] = df_consolidated['pull_requests_merged_count']

# Recalculate 'Code Changes' using scaled additions and deletions and include the suffix
df_consolidated[f'Code Changes{scaled_label_suffix}'] = df_consolidated['scaled_additions'] + df_consolidated['scaled_deletions']

# Define the new consolidated event columns for plotting, now including the scaled suffix
consolidated_event_columns = ['Issues', 'Comments', 'PRs Opened', 'PRs Merged', f'Code Changes{scaled_label_suffix}']

# Filter out developers with no activity in any of the consolidated events
df_plot_consolidated = df_consolidated[df_consolidated[consolidated_event_columns].sum(axis=1) > 0].copy()

# Melt the DataFrame to a long format suitable for stacked bar charts
df_melted_consolidated = df_plot_consolidated.melt(
    id_vars=['github_login', 'name'],
    value_vars=consolidated_event_columns,
    var_name='Event Type',
    value_name='Count'
)

# Rename 'Issues' to 'Issues Raised' for display in the plot legend
df_melted_consolidated['Event Type'] = df_melted_consolidated['Event Type'].replace('Issues', 'Issues Raised')

# Add a new column for text display, conditional on Count > 0 and formatted
df_melted_consolidated['text_values'] = df_melted_consolidated.apply(lambda row: f"{row['Count']:.2f}" if row['Count'] > 0 and row['Event Type'] == f'Code Changes{scaled_label_suffix}' else (f"{row['Count']:.0f}" if row['Count'] > 0 else ''), axis=1)


# Create the stacked bar chart using Plotly Express
fig_consolidated = px.bar(
    df_melted_consolidated,
    x='github_login',
    y='Count',
    color='Event Type',
    title=f'Aggregate Event count per Contributor (Scaled Code Changes{scaled_label_suffix})', # Updated title
    labels={'github_login': 'Contributor', 'Count': 'Event Count'},
    hover_name='name',
    hover_data={'Event Type': True, 'Count': ':.2f', 'github_login': False}, # Format Count to 2 decimal places
    text='text_values' # Use the new conditional text column
)

fig_consolidated.update_layout(
    xaxis_title='Contributor',
    yaxis_title='Event Count',
    barmode='stack',
    legend_title='Event Type'
)

# Adjust x-axis to show labels vertically
fig_consolidated.update_xaxes(tickangle=90, tickfont=dict(size=10), showgrid=False)

# Revert y-axis grid to default Plotly style, with lines every 5 units
fig_consolidated.update_yaxes(showgrid=True, showticklabels=True, dtick=5, gridwidth=1, griddash=None)

# Set text color to white for all traces and position text inside
fig_consolidated.update_traces(textfont_color='white', textposition='inside')

fig_consolidated.show()

save_fig(fig_consolidated, 'all_aggregate_events.html')

Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/all_aggregate_events.html


## Inactive developers

* Devs that didn't contribute dureing the period

In [26]:
import pandas as pd
from IPython.display import display, Markdown

# Define the columns that represent contribution events
contribution_columns = [
    'issues_opened_count',
    'issues_reopened_count',
    'issue_comment_event_count',
    'pull_requests_opened_count',
    'pull_requests_merged_count',
    'pull_requests_closed_count',
    'total_additions',
    'total_deletions'
]

# Ensure all contribution columns exist in df_people
existing_contribution_columns = [col for col in contribution_columns if col in df_people.columns]

# Calculate the total contributions for each person
# Summing across relevant columns for each row
df_people['total_contributions'] = df_people[existing_contribution_columns].sum(axis=1)

# Filter for developers with zero total contributions
inactive_devs_df = df_people[df_people['total_contributions'] == 0]

# Extract logins of inactive developers for potential future use
inactive_dev_logins = inactive_devs_df['github_login'].tolist()

if not inactive_devs_df.empty:
    display(Markdown(f"**{len(inactive_devs_df)} Developer(s) with No Contributions in the Period:**"))
    for index, row in inactive_devs_df.iterrows():
        dev_name = row['name'] if row['name'] != 'N/A' else 'Name not available'
        display(Markdown(f"- {row['github_login']} ({dev_name})"))
else:
    display(Markdown("**All developers have made at least one contribution in the selected period!**"))

# Clean up the temporary column
df_people.drop(columns=['total_contributions'], inplace=True, errors='ignore')


**40 Developer(s) with No Contributions in the Period:**

- chonkachu (Name not available)

- fernandovtorres (Fernando Valentim Torres)

- bdogabriel (Gabriel Barbosa)

- liestrela (li)

...

- muriloleandro (Murilo Leandro)

- arturkenzo (Artur Kenzo Obara Kawazoe)

- gustavorspires (Gustavo Ramos Santos Pires)

- Marcelo-MConti (Name not available)

- Misterio77 (Gab Fontes)

## Workload balance (WB) index

In [27]:
# cellname : wbi
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Calculate aggregate PRs opened and closed for the entire project
total_prs_opened = df_people['pull_requests_opened_count'].sum()
total_prs_closed = df_people['pull_requests_closed_count'].sum()

print(f"Total PRs Opened in the period: {total_prs_opened}")
print(f"Total PRs Closed in the period: {total_prs_closed}")

# --- Calculate the NEW Aggregate Progress Index (PI) (formerly PR Balance Index) ---
# PI = (PRs Closed - PRs Opened) / (PRs Closed + PRs Opened)
# Handle case where total_pr_activity is 0 to avoid division by zero
total_pr_activity = total_prs_opened + total_prs_closed

if total_pr_activity > 0:
    # The new PI is the former PR Balance Index
    wb_index = (total_prs_closed - total_prs_opened) / total_pr_activity
    print(f"Work Balance (WB): {wb_index:.2f}\n")
else:
    wb_index = 0.0 # Define a default if no PRs were opened and closed
    print("No PR activity (opened or closed) in the period to calculate the Net Contribution Throughput (NCT).\n")

# --- Horizontal Bar Chart for Net Contribution Throughput (NCT) ---

# Determine marker color dynamically
#bar_color_pi = '#AEC6CF'
if wb_index < 0:
    bar_color_pi = "#dd5500"
elif wb_index > 0:
    bar_color_pi = "#00aa88"

fig_horizontal_bar = go.Figure()

# Add background shapes for the ranges (reverting y0 and y1 for taller colored segments)
fig_horizontal_bar.update_layout(
    shapes=[
        # More vivid Red for [-1, -0.5]
        dict(type='rect', xref='x', yref='y', x0=-1, y0=-0.5, x1=-0.5, y1=0.5, fillcolor='#FF6347', opacity=0.3, layer='below', line_width=0),
        # More vivid Salmon for [-0.5, 0]
        dict(type='rect', xref='x', yref='y', x0=-0.5, y0=-0.5, x1=0, y1=0.5, fillcolor='#FFA07A', opacity=0.3, layer='below', line_width=0),
        # More vivid Light Green for [0, 0.5]
        dict(type='rect', xref='x', yref='y', x0=0, y0=-0.5, x1=0.5, y1=0.5, fillcolor='#90EE90', opacity=0.3, layer='below', line_width=0),
        # More vivid Darker Green for [0.5, 1]
        dict(type='rect', xref='x', yref='y', x0=0.5, y0=-0.5, x1=1, y1=0.5, fillcolor='#3CB371', opacity=0.3, layer='below', line_width=0)
    ]
)

fig_horizontal_bar.add_trace(go.Bar(
    x=[wb_index],
    y=['NCT'],
    orientation='h',
    base=0,
    marker_color=bar_color_pi,
    text=f'{wb_index:.2f}',
    textposition='inside',
    textfont=dict(color='white'),
    width=0.4 # Explicitly setting the width (height for horizontal bar) of the blue indicator bar
))

fig_horizontal_bar.update_layout(
    title_text='Workload Balance (WB)', # Updated title
    xaxis_range=[-1, 1], # Set x-axis range strictly from -1 to 1
    xaxis_title='← Accumulating | Clearing →', # Updated x-axis title
    yaxis_visible=False, # Hide y-axis
    yaxis_showticklabels=False, # Hide y-axis tick labels
    height=150, # Set a reasonable height for the bar chart
    width=800, # Set a reasonable width
    margin=dict(l=20, r=20, t=50, b=20), # Adjust margins
    title_x=0.5 # Center the title
)

fig_horizontal_bar.show()

save_fig(fig_horizontal_bar, 'workload_balance.html')


Total PRs Opened in the period: 11
Total PRs Closed in the period: 12
Work Balance (WB): 0.04



Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/workload_balance.html


## Execution balance index (EBI)

In [28]:
# cellname : ebi

import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Calculate total issues raised (opened + reopened)
total_issues_raised = df_people['issues_opened_count'].sum() + df_people['issues_reopened_count'].sum()

# Get total PRs submitted (using 'pull_requests_opened_count' as proxy for submitted)
total_prs_submitted = df_people['pull_requests_opened_count'].sum()

print(f"Total Issues Raised in the period: {total_issues_raised}")
print(f"Total PRs Submitted in the period: {total_prs_submitted}")

# --- Calculate the Submitted Index (SI) ---
# SI = (PRs Submitted - Issues Raised) / (PRs Submitted + Issues Raised)
# Handle case where total_activity is 0 to avoid division by zero
total_activity_for_si = total_prs_submitted + total_issues_raised

if total_activity_for_si > 0:
    rb_index = (total_prs_submitted - total_issues_raised) / total_activity_for_si
    print(f"Resolution Balance (RB): {rb_index:.2f}\n")
else:
    rb_index = 0.0 # Define a default if no relevant activity
    print("No PRs submitted or issues raised in the period to calculate the Submitted Index (SI).\n")

# --- Horizontal Bar Chart for Submitted Index (SI) ---

# Determine indicator bar color based on index value, as originally designed
if rb_index < 0:
    bar_color_si = "#888800"
elif rb_index > 0:
    bar_color_si = "#0088aa"
else:
    bar_color_si = "#AEC6CF" # Default grayish-blue if index is 0

fig_submitted_bar = go.Figure()

# Add background shapes for the ranges (reusing vivid colors from previous notebook state)
fig_submitted_bar.update_layout(
    shapes=[
        # More vivid Red for [-1, -0.5] -> Changed to Yellow
        dict(type='rect', xref='x', yref='y', x0=-1, y0=-0.5, x1=-0.5, y1=0.5, fillcolor='#FFDD33', opacity=0.3, layer='below', line_width=0),
        # More vivid Salmon for [-0.5, 0] -> Changed to Lighter Yellow
        dict(type='rect', xref='x', yref='y', x0=-0.5, y0=-0.5, x1=0, y1=0.5, fillcolor='#FFEE99', opacity=0.3, layer='below', line_width=0),
        # More vivid Light Green for [0, 0.5] -> Changed to Light Blue
        dict(type='rect', xref='x', yref='y', x0=0, y0=-0.5, x1=0.5, y1=0.5, fillcolor='#ADD8E6', opacity=0.3, layer='below', line_width=0),
        # More vivid Darker Green for [0.5, 1] -> Changed to Darker Blue
        dict(type='rect', xref='x', yref='y', x0=0.5, y0=-0.5, x1=1, y1=0.5, fillcolor='#4682B4', opacity=0.3, layer='below', line_width=0)
    ]
)

fig_submitted_bar.add_trace(go.Bar(
    x=[rb_index],
    y=['SI'],
    orientation='h',
    base=0,
    marker_color=bar_color_si,
    text=f'{rb_index:.2f}',
    textposition='inside',
    textfont=dict(color='white'), # White text color
    width=0.4 # Height of the indicator bar
))

fig_submitted_bar.update_layout(
    title_text='Resolution Balance (RB)', # Updated title
    xaxis_range=[-1, 1], # Set x-axis range strictly from -1 to 1
    xaxis_title='← Planning | Implementing →', # Updated x-axis title
    yaxis_visible=False, # Hide y-axis
    yaxis_showticklabels=False, # Hide y-axis tick labels
    height=150, # Set a reasonable height for the bar chart
    width=800, # Set a reasonable width
    margin=dict(l=20, r=20, t=50, b=20), # Adjust margins
    title_x=0.5 # Center the title
)

fig_submitted_bar.show()

save_fig(fig_submitted_bar, 'resolution_balance.html')


Total Issues Raised in the period: 26
Total PRs Submitted in the period: 11
Resolution Balance (RB): -0.41



Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/resolution_balance.html


## Develoment tide chart : WB x RB

In [29]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# 2. Create a Pandas DataFrame named df_metrics with 'EB' (submitted_index) and 'WB' (aggregate_pi)
# Ensure rb_index and wb_index are available from previous cells
df_metrics = pd.DataFrame({
    'RB': [rb_index],
    'WB': [wb_index]
})

# 1. Calculate the polar coordinates r (radius) and theta_degrees (angle in degrees)
#    from the EB and WB values in df_metrics.
df_metrics['r'] = np.sqrt(df_metrics['RB']**2 + df_metrics['WB']**2)
df_metrics['theta_degrees'] = np.degrees(np.arctan2(df_metrics['RB'], df_metrics['WB']))

# 2. Initialize the figure
fig = go.Figure()

# Add a separate trace for the line from the origin to the point (added first to be in the background)
fig.add_trace(go.Scatterpolar(
    r=[0, df_metrics['r'].iloc[0]], # Draw line from origin (r=0) to the point's r
    theta=[df_metrics['theta_degrees'].iloc[0], df_metrics['theta_degrees'].iloc[0]], # Maintain same angle
    mode='lines',
    line=dict(width=1, color='black', dash='dot'), # Updated line style: thin, black, dotted
    name='Handle from Origin'
))

# Add the trace for the marker and its text (added second to be on top)
fig.add_trace(go.Scatterpolar(
    r=df_metrics['r'],
    theta=df_metrics['theta_degrees'],
    mode='markers+text', # Restored mode to markers+text
    marker=dict(size=15, color='blue'),
    text=[f'RB: {df_metrics['RB'].iloc[0]:.2f}<br>WB: {df_metrics['WB'].iloc[0]:.2f}'],
    textposition='top center',
    name='Current State'
))

# 3. Update the layout of the figure to configure the polar axes
fig.update_layout(
    title_text='Development tide chart (WB x RB)',
    title_x=0.5,
    height=600,
    width=800,
    polar=dict(
        radialaxis=dict(range=[0, 1.1]),
        angularaxis=dict(
            rotation=90, # To align 0 degrees with 'Up' (+WB)
            direction='clockwise', # For clockwise angle increase
            tickvals=[0, 90, 180, 270],
            ticktext=['Clearing (+WB)', 'Implementing (+RB)', 'Accumulating (-WB)', 'Planning (-RB)']
        )
    ),
    showlegend=False # Hide legend as it's a single point and handle
)

# 6. Display the plot
fig.show()

save_fig(fig, 'wb_rb_map.html')


Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/wb_rb_map.html


## PR completion-time in the period

In [30]:
# cellname : pr-close-time

from datetime import datetime, timezone
import pandas as pd
import plotly.express as px

# Ensure selected_timedelta and repo are available from previous cells
end_date = datetime.now(timezone.utc)
start_date = end_date - selected_timedelta

pr_close_times = []

# Initialize counters for the new categories
opened_before_closed_in_period_count = 0
opened_and_closed_in_period_count = 0
opened_in_period_still_open_count = 0
opened_before_still_open_count = 0

print(f"Analyzing PRs relative to the period: {start_date.strftime('%Y-%m-%d %H:%M:%S')} to {end_date.strftime('%Y-%m-%d %H:%M:%S')}...")

# Fetch ALL pull requests (both open and closed) for comprehensive analysis
all_pulls = repo.get_pulls(state='all')

for pull in all_pulls:
    # Category: Opened before period and closed within period
    if pull.created_at < start_date and pull.closed_at and start_date <= pull.closed_at <= end_date:
        opened_before_closed_in_period_count += 1

    # Category: Opened and closed within period
    if start_date <= pull.created_at <= end_date and pull.closed_at and start_date <= pull.closed_at <= end_date:
        opened_and_closed_in_period_count += 1
        # Also add to pr_close_times for detailed analysis and histogram
        time_to_close = pull.closed_at - pull.created_at
        pr_close_times.append({
            'number': pull.number,
            'title': pull.title,
            'creator': pull.user.login if pull.user else 'N/A',
            'created_at': pull.created_at,
            'closed_at': pull.closed_at,
            'time_to_close_days': time_to_close.total_seconds() / (24 * 3600)
        })

    # Category: Opened in period and not yet closed
    if start_date <= pull.created_at <= end_date and pull.closed_at is None:
        opened_in_period_still_open_count += 1

    # Category: Opened before period and still open
    if pull.created_at < start_date and pull.closed_at is None:
        opened_before_still_open_count += 1

print("\n--- PR Categorization for the Period ---")
print(f"* PRs opened before period and closed in period: {opened_before_closed_in_period_count}")
print(f"* PRs opened and closed in period: {opened_and_closed_in_period_count}")
print(f"* PRs opened in period and not closed: {opened_in_period_still_open_count}")
print(f"* PRs opened before period and still open: {opened_before_still_open_count}")

df_pr_close_times = pd.DataFrame(pr_close_times)

if not df_pr_close_times.empty:
    print(f"\nFound {len(df_pr_close_times)} PRs opened and closed within the period for detailed analysis.")
    display(df_pr_close_times.head())

    print("\n--- Summary of PR Close Times (Opened & Closed in Period) ---")
    print(f"Average time to close PR: {df_pr_close_times['time_to_close_days'].mean():.2f} days")
    print(f"Median time to close PR: {df_pr_close_times['time_to_close_days'].median():.2f} days")
    print(f"Minimum time to close PR: {df_pr_close_times['time_to_close_days'].min():.2f} days")
    print(f"Maximum time to close PR: {df_pr_close_times['time_to_close_days'].max():.2f} days")

    # Optional: Plotting a histogram of close times
    fig = px.histogram(df_pr_close_times, x='time_to_close_days', nbins=10,
                       title='PR completion-time histogram (in the period)',
                       labels={'time_to_close_days': 'Time to Close (Days)', 'count': 'Number of PRs'})
    fig.show()
    save_fig(fig, 'pr_close_times_histogram.html')
else:
    print("\nNo PRs were opened and closed within the selected period for detailed time analysis.")

Analyzing PRs relative to the period: 2025-11-19 22:55:13 to 2025-11-26 22:55:13...

--- PR Categorization for the Period ---
* PRs opened before period and closed in period: 4
* PRs opened and closed in period: 8
* PRs opened in period and not closed: 4
* PRs opened before period and still open: 6

Found 8 PRs opened and closed within the period for detailed analysis.


,number,title,creator,created_at,closed_at,time_to_close_days
0,423,[FEAT] rainbow snake,trqt,2025-11-26 15:12:42+00:00,2025-11-26 20:47:29+00:00,0.232488
1,421,[FEAT] Teleport Mode,JuanHPassos,2025-11-26 13:56:20+00:00,2025-11-26 19:13:26+00:00,0.220208
2,418,[FEAT] Cheese Mode,DeguShi,2025-11-25 23:59:37+00:00,2025-11-26 02:38:22+00:00,0.110243
3,417,docs: add code of conduct,otaviobia,2025-11-25 23:54:34+00:00,2025-11-26 17:51:03+00:00,0.747558
4,412,Fix: change the import,JuanHPassos,2025-11-25 22:54:13+00:00,2025-11-25 22:56:38+00:00,0.001678



--- Summary of PR Close Times (Opened & Closed in Period) ---
Average time to close PR: 0.39 days
Median time to close PR: 0.23 days
Minimum time to close PR: 0.00 days
Maximum time to close PR: 1.48 days


Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/pr_close_times_histogram.html


In [31]:
import plotly.graph_objects as go
import pandas as pd

# Ensure the df_pr_close_times DataFrame is not empty
if not df_pr_close_times.empty:
    # Create the scatter plot trace
    scatter_trace = go.Scatter(
        x=df_pr_close_times['number'],
        y=df_pr_close_times['time_to_close_days'],
        mode='markers',
        marker=dict(size=10, opacity=0.8, line=dict(width=1, color='DarkSlateGrey')),
        hoverinfo='text',
        # Custom hover text to display detailed PR information
        text=df_pr_close_times.apply(lambda row:
            f"PR #{row['number']}<br>" +
            f"Title: {row['title']}<br>" +
            f"Creator: {row['creator']}<br>" +
            f"Created: {row['created_at'].strftime('%Y-%m-%d %H:%M:%S')}<br>" +
            f"Closed: {row['closed_at'].strftime('%Y-%m-%d %H:%M:%S')}<br>" +
            f"Time to Close: {row['time_to_close_days']:.2f} days", axis=1)
    )

    # Create the figure and add the trace
    fig = go.Figure(data=[scatter_trace])

    # Calculate quartiles
    q1 = df_pr_close_times['time_to_close_days'].quantile(0.25)
    median = df_pr_close_times['time_to_close_days'].quantile(0.50)
    q3 = df_pr_close_times['time_to_close_days'].quantile(0.75)

    # Add horizontal lines for quartiles
    fig.add_hline(y=q1, line_dash='dash', line_color='green', line_width=1, annotation_text=f"Q1: {q1:.2f} days", annotation_position="bottom right")
    fig.add_hline(y=median, line_dash='dash', line_color='blue', line_width=1, annotation_text=f"Median: {median:.2f} days", annotation_position="bottom right")
    fig.add_hline(y=q3, line_dash='dash', line_color='red', line_width=1, annotation_text=f"Q3: {q3:.2f} days", annotation_position="bottom right")

    # Update layout for title and axis labels
    fig.update_layout(
        title='PR completion time (in the perio) with quartiles',
        xaxis_title='Pull Request Number',
        yaxis_title='Time to Close (Days)',
        height=600,
        hovermode='closest' # Show hover info for the closest point
    )

    # Display the plot
    fig.show()
    save_fig(fig, 'pr_close_time_scatter_plot.html')
else:
    print("No PRs were opened and closed within the selected period to plot.")

Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/pr_close_time_scatter_plot.html


In [32]:
import plotly.express as px
import pandas as pd

# Data from the previous cell's execution
pr_category_data = {
    'Category': [
        'Opened before period & closed in period',
        'Opened & closed in period',
        'Opened in period & not closed',
        'Opened before period & still open'
    ],
    'Count': [
        opened_before_closed_in_period_count,
        opened_and_closed_in_period_count,
        opened_in_period_still_open_count,
        opened_before_still_open_count
    ]
}

df_pr_categories = pd.DataFrame(pr_category_data)

# Filter out categories with 0 count to avoid empty slices
df_pr_categories = df_pr_categories[df_pr_categories['Count'] > 0]

if df_pr_categories.empty:
    print("No PR activity found to categorize for the pie chart.")
else:
    fig = px.pie(
        df_pr_categories,
        values='Count',
        names='Category', # Use the raw category name for slice identification
        title='Distribution of PR status in the period',
        hole=0.3, # Optional: creates a donut chart
        color_discrete_sequence=px.colors.qualitative.Pastel # Use a pastel color sequence
    )

    fig.update_traces(
        textposition='outside',
        textinfo='percent+label+value',
        textfont_color='black'
        # Optional: rotate labels if they overlap
        # insidetextorientation='radial'
    )
    fig.update_layout(
        showlegend=True,
        legend=dict(
            orientation='h', # Horizontal orientation for bottom placement
            yanchor='bottom',   # Anchor to the bottom
            y=-0.3,             # Position further below the plot area
            xanchor='center',
            x=0.5              # Center horizontally
        )
    )
    fig.show()
    save_fig(fig, 'pr_categories_pie_chart.html')


Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/pr_categories_pie_chart.html


# DEVELOPERS OVERVIEW

## Engagement

### Engament chart

In [33]:
# cellname : engagement-plot

import plotly.express as px
import pandas as pd

# Calculate the 'Discussion' metric
df_plot_discussion = df_people.copy()
df_plot_discussion['Discussion'] = df_plot_discussion['issues_opened_count'] + df_plot_discussion['issue_comment_event_count']

# Filter out developers who have no activity in either PRs Opened or Discussion
df_plot_discussion = df_plot_discussion[
    (df_plot_discussion['pull_requests_opened_count'] > 0) |
    (df_plot_discussion['Discussion'] > 0)
].copy()

# --- Aggregate data for unique (PRs Opened, Discussion) coordinates ---
aggregated_df = df_plot_discussion.groupby(['pull_requests_opened_count', 'Discussion']).agg(
    github_logins=('github_login', lambda x: ', '.join(x)),
    names=('name', lambda x: ', '.join(x)),
    num_devs=('github_login', 'count')
).reset_index()

# Create a unique key for each aggregated point for coloring with distinct discrete colors
# This key will also be used initially as the legend name before being replaced by logins
aggregated_df['point_category'] = 'PRs ' + aggregated_df['pull_requests_opened_count'].astype(str) + ', Discussion ' + aggregated_df['Discussion'].astype(str)

# Add a new column to aggregated_df to determine marker style
aggregated_df['marker_type'] = aggregated_df['num_devs'].apply(lambda x: 'Single Contributor' if x == 1 else 'Multiple Contributors')

# Define a custom color sequence using only Dark24
custom_color_sequence = px.colors.qualitative.Dark24

# Create the scatter plot with aggregated data
fig = px.scatter(
    aggregated_df,
    x='pull_requests_opened_count',
    y='Discussion',
    size='num_devs', # Size of marker based on number of developers at this point
    color='point_category', # Use the categorical key for distinct colors
    color_discrete_sequence=custom_color_sequence, # Use the custom, extended palette
    symbol='marker_type', # Use the new column to determine marker symbol (solid vs. outlined)
    symbol_map={'Single Contributor': 'circle', 'Multiple Contributors': 'circle-open'}, # Explicitly map styles
    text=None, # Removed text labels from directly on the bubbles
    title='Engagement chart',
    labels={
        'pull_requests_opened_count': 'PRs Opened',
        'Discussion': 'Discussion (Issues Opened + Comments)',
        'num_devs': 'Number of Contributors',
        'github_logins': 'Contributors',
        'point_category': 'Contributors' # This will be the initial legend title, replaced later
    },
    hover_name='names', # Show aggregated names on hover
    hover_data={
        'github_logins': True, # Also show aggregated logins on hover
        'num_devs': True,
        'pull_requests_opened_count': True,
        'Discussion': True,
        'names': False,
        'point_category': False
    }
)

# The update_traces for textposition will now do nothing as text=None
# fig.update_traces(textposition='middle right', textfont_size=10)

# Increase border thickness for 'circle-open' symbols, inheriting color
fig.update_traces(marker=dict(line=dict(width=8, color='black')), selector=dict(symbol='circle-open'))

# Create a mapping from point_category to github_logins for legend renaming
category_to_logins_map = aggregated_df.set_index('point_category')['github_logins'].to_dict()

# Function to extract the base point_category from the full trace name
def get_base_point_category(full_trace_name):
    # The trace name will be 'point_category_value, marker_type_value' if both color and symbol are used
    # We want to split at the last comma to get 'point_category_value'
    parts = full_trace_name.rsplit(', ', 1)
    if len(parts) > 1 and (parts[-1] == 'Single Contributor' or parts[-1] == 'Multiple Contributors'):
        return parts[0]
    return full_trace_name # Fallback if name doesn't match expected pattern (e.g., if only color is used)

# Update the legend entry names to show github_logins
fig.for_each_trace(lambda trace: trace.update(name=category_to_logins_map[get_base_point_category(trace.name)]))

fig.update_layout(
    height=1200, # Increased height to accommodate the legend
    xaxis_title='PRs Opened',
    yaxis_title='Discussion (Issues Opened + Comments)',
    legend_title='Contributors',
    legend=dict(
        orientation='v', # Vertical orientation
        yanchor='top', # Anchor to the top
        y=-0.2, # Position below the chart (adjust as needed)
        xanchor='left',
        x=0
    )
)

fig.show()

save_fig(fig, 'engagement_plot.html')

Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/engagement_plot.html


### Engagement magnitude (EM) index

In [34]:
# cellname : engagement-index

import plotly.express as px
import pandas as pd
import numpy as np

# Calculate the 'Discussion' metric if not already present
df_bci = df_people.copy()
df_bci['Discussion'] = df_bci['issues_opened_count'] + df_bci['issue_comment_event_count']

# Filter out developers with BCI equal to 0 (no contributions in these categories)
df_em_plot = df_bci[((df_bci['Discussion'] > 0) | (df_bci['pull_requests_opened_count'] > 0))].copy()

# --- Normalize coordinates (Discussion, PRs Opened) between 0 and 1 ---
max_discussion = df_em_plot['Discussion'].max()
max_prs_opened = df_em_plot['pull_requests_opened_count'].max()

df_em_plot['Normalized_Discussion'] = 0
if max_discussion > 0:
    df_em_plot['Normalized_Discussion'] = (df_em_plot['Discussion'] / max_discussion)

df_em_plot['Normalized_PRs_Opened'] = 0
if max_prs_opened > 0:
    df_em_plot['Normalized_PRs_Opened'] = (df_em_plot['pull_requests_opened_count'] / max_prs_opened)

# Calculate the Engagement Magnitude Index (EM) using normalized values
# EM = sqrt(Normalized_Discussion^2 + Normalized_PRs_Opened^2)
df_em_plot['EM'] = np.sqrt(df_em_plot['Normalized_Discussion']**2 + df_em_plot['Normalized_PRs_Opened']**2)

# Normalize EM by its theoretical maximum of sqrt(2) to scale it between 0 and 1
if not df_em_plot['EM'].empty:
    df_em_plot['EM'] = df_em_plot['EM'] / np.sqrt(2)

# Sort the DataFrame by EM in descending order
df_em_plot = df_em_plot.sort_values(by='EM', ascending=False)

# Create the bar chart for EM
fig = px.bar(
    df_em_plot,
    x='github_login',
    y='EM',
    title='Engagement magnitude (EM) index',
    labels={
        'github_login': 'Contributor',
        'EM': 'EM'
    },
    hover_name='name',
    hover_data={
        'github_login': False,
        'name': True,
        'Discussion': True,
        'pull_requests_opened_count': True,
        'Normalized_Discussion': ':.2f',
        'Normalized_PRs_Opened': ':.2f',
        'EM': ':.2f' # Format EM to 2 decimal places in hover
    },
    text_auto='.1f' # Format EM to 1 decimal place on top of bars
)

# Adjust x-axis to show labels vertically
fig.update_xaxes(tickangle=90, tickfont=dict(size=10))
fig.update_yaxes(rangemode='tozero') # Ensure y-axis starts at zero

# Optional: adjust text position if needed (default 'auto' for bar usually puts it on top)
fig.update_traces(textposition='outside')

fig.show()

save_fig(fig, 'em_index.html')


Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/em_index.html


REMI is the norm of the two-dimensional contribution vector whose coordinates are the developer’s Discussion score and the number of PRs opened. In practice, EMI captures the overall intensity of a developer’s participatory and initiatory actions by combining conversational engagement with concrete technical initiation into a single magnitude.

## Delivery

### Delivery chart

In [35]:
# cellname : deliery-plot

import plotly.express as px
import pandas as pd

# Calculate the 'Code Changes' metric
df_plot_code_changes = df_people.copy()
df_plot_code_changes['Code Changes'] = df_plot_code_changes['total_additions'] + df_plot_code_changes['total_deletions']

# Filter out developers who have no activity in either PRs Merged or Code Changes
df_plot_code_changes = df_plot_code_changes[
    (df_plot_code_changes['pull_requests_merged_count'] > 0) |
    (df_plot_code_changes['Code Changes'] > 0)
].copy()

# --- Aggregate data for unique (PRs merged, Code Changes) coordinates ---
aggregated_df = df_plot_code_changes.groupby(['pull_requests_merged_count', 'Code Changes']).agg(
    github_logins=('github_login', lambda x: ', '.join(x)),
    names=('name', lambda x: ', '.join(x)),
    num_devs=('github_login', 'count')
).reset_index()

# Create a unique key for each aggregated point for coloring with distinct discrete colors
aggregated_df['point_category'] = 'PRs Merged ' + aggregated_df['pull_requests_merged_count'].astype(str) + ', Changes ' + aggregated_df['Code Changes'].astype(str)

# Add a new column to aggregated_df to determine marker style
# Updated: 'circle' for single, 'circle-open' for multiple. 'circle-dot' would be third if needed.
aggregated_df['marker_type'] = aggregated_df['num_devs'].apply(lambda x: 'Single Contributor' if x == 1 else 'Multiple Contributors')

# Define a custom color sequence using only Dark24
custom_color_sequence = px.colors.qualitative.Dark24

# Create the scatter plot with aggregated data (axes swapped)
fig = px.scatter(
    aggregated_df,
    x='Code Changes', # Swapped axis
    y='pull_requests_merged_count', # Swapped axis
    size='num_devs', # Size of marker based on number of developers at this point
    color='point_category', # Use the categorical key for distinct colors
    color_discrete_sequence=custom_color_sequence, # Use the custom, extended palette
    symbol='marker_type', # Use the new column to determine marker symbol (solid vs. outlined)
    symbol_map={'Single Contributor': 'circle', 'Multiple Contributors': 'circle-open'}, # Updated symbol_map
    text=None, # Removed text labels from directly on the bubbles
    title='Delivery chart',
    labels={
        'Code Changes': 'Code Changes (Additions + Deletions)', # Updated label
        'pull_requests_merged_count': 'PRs Merged', # Updated label
        'num_devs': 'Number of Contributors',
        'github_logins': 'Contributors',
        'point_category': 'Contributors' # This will be the initial legend title, replaced later
    },
    hover_name='names', # Show aggregated names on hover
    hover_data={
        'github_logins': True, # Also show aggregated logins on hover
        'num_devs': True,
        'pull_requests_merged_count': True,
        'Code Changes': True,
        'names': False, # Don't duplicate hover_name in hover_data
        'point_category': False # Don't show this in hover data, as labels are clear
    }
)

# Apply marker styling to ensure proportionality with small base size
# sizeref: smaller value makes markers larger. Larger value makes markers smaller.
# Let's target a max size of ~10 pixels for the largest 'num_devs' (reduced from 20)
max_num_devs_val = aggregated_df['num_devs'].max()
if max_num_devs_val > 0:
    target_max_size = 10 # pixels for the largest bubble (reduced)
    sizeref_val = max_num_devs_val / target_max_size
else:
    sizeref_val = 1 # avoid division by zero if all num_devs are 0 or 1

fig.update_traces(
    marker=dict(
        sizemode='diameter', # Scale by diameter
        sizeref=sizeref_val, # Reference value for scaling
        sizemin=2,           # Minimum size in pixels for the smallest 'num_devs' (reduced from 4)
        line=dict(width=1, color='DarkSlateGrey') # Default line for all
    ),
    selector=dict(mode='markers') # Apply to all scatter markers
)

# If specific line widths for symbols are desired, they can be added here
# For example, to make the 'circle-open' slightly thicker outline
fig.update_traces(marker=dict(line=dict(width=2)), selector=dict(symbol='circle-open'))


# Create a mapping from point_category to github_logins for legend renaming
category_to_logins_map = aggregated_df.set_index('point_category')['github_logins'].to_dict()

# Function to extract the base point_category from the full trace name
def get_base_point_category(full_trace_name):
    # The trace name will be 'point_category_value, marker_type_value' if both color and symbol are used
    # We want to split at the last comma to get 'point_category_value'
    parts = full_trace_name.rsplit(', ', 1)
    if len(parts) > 1 and (parts[-1] == 'Single Contributor' or parts[-1] == 'Multiple Contributors'):
        return parts[0]
    return full_trace_name # Fallback if name doesn't match expected pattern (e.g., if only color is used)

# Update the legend entry names to show github_logins
fig.for_each_trace(lambda trace: trace.update(name=category_to_logins_map[get_base_point_category(trace.name)]))

fig.update_layout(
    height=800, # Reduced height
    xaxis_title='Code Changes (Additions + Deletions)', # Swapped axis title
    yaxis_title='PRs Merged',
    legend_title='Contributors',
    legend=dict(
        orientation='v', # Vertical orientation
        yanchor='top', # Anchor to the top
        y=-0.2, # Position below the chart (adjust as needed)
        xanchor='left',
        x=0
    )
)

fig.show()

save_fig(fig, 'delivery_plot.html')


Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/delivery_plot.html


### Delivery magnitude (DM) index




In [36]:
# cellname : delivery-index
import plotly.express as px
import pandas as pd
import numpy as np

# Calculate the 'Code Changes' metric if not already present
df_dm = df_people.copy()
df_dm['Code Changes'] = df_dm['total_additions'] + df_dm['total_deletions']

# Filter out developers with no contributions in these categories
df_dm_plot = df_dm[((df_dm['Code Changes'] > 0) | (df_dm['pull_requests_merged_count'] > 0))].copy()

# --- Normalize coordinates (Code Changes, PRs Merged) between 0 and 1 ---
max_code_changes = df_dm_plot['Code Changes'].max()
max_prs_merged = df_dm_plot['pull_requests_merged_count'].max()

df_dm_plot['Normalized_Code_Changes'] = 0
if max_code_changes > 0:
    df_dm_plot['Normalized_Code_Changes'] = (df_dm_plot['Code Changes'] / max_code_changes)

df_dm_plot['Normalized_PRs_Merged'] = 0
if max_prs_merged > 0:
    df_dm_plot['Normalized_PRs_Merged'] = (df_dm_plot['pull_requests_merged_count'] / max_prs_merged)

# Calculate the Delivery Magnitude Index (DM) using normalized values
# DM = sqrt(Normalized_Code_Changes^2 + Normalized_PRs_Merged^2)
df_dm_plot['DM'] = np.sqrt(df_dm_plot['Normalized_Code_Changes']**2 + df_dm_plot['Normalized_PRs_Merged']**2)

# Normalize the final index value so that it fits in [0, 1]
# The maximum possible value for DM is sqrt(1^2 + 1^2) = sqrt(2)
if not df_dm_plot['DM'].empty:
    df_dm_plot['DM'] = df_dm_plot['DM'] / np.sqrt(2)

# Sort the DataFrame by DM in descending order
df_dm_plot = df_dm_plot.sort_values(by='DM', ascending=False)

# Create the bar chart for DM
fig = px.bar(
    df_dm_plot,
    x='github_login',
    y='DM',
    title='Delivery magnitude (DM) per contributor',
    labels={
        'github_login': 'Contributor',
        'DM': 'DM'
    },
    hover_name='name',
    hover_data={
        'github_login': False,
        'name': True,
        'Code Changes': True,
        'pull_requests_merged_count': True,
        'Normalized_Code_Changes': ':.2f',
        'Normalized_PRs_Merged': ':.2f',
        'DM': ':.2f' # Format to 2 decimal places in hover
    },
    text_auto='.1f' # Format to 1 decimal place on top of bars
)

# Adjust x-axis to show labels vertically
fig.update_xaxes(tickangle=90, tickfont=dict(size=10))
fig.update_yaxes(rangemode='tozero') # Ensure y-axis starts at zero

# Adjust text position
fig.update_traces(textposition='outside')

fig.show()

save_fig(fig, 'dm_index.html')


Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/dm_index.html


The Relative Delivery Magnitude Index (RDMI) is the norm of the two-dimensional delivery vector whose components are the developer’s code changes and the number of PRs merged. Conceptually, CDI represents the overall strength of a developer’s effective code delivery by combining the volume of code produced with the number of contributions successfully integrated into the project.

## Engagement-Delivery Map

In [37]:
import plotly.express as px
import pandas as pd

# Ensure df_em_plot and df_dm_plot are available and contain the calculated indices
# If this cell is run independently, ensure these DFs are regenerated or loaded

# Select relevant columns from each DataFrame
df_em = df_em_plot[['github_login', 'name', 'EM']]
df_dm = df_dm_plot[['github_login', 'name', 'DM']]

# Merge the two DataFrames on 'github_login'
# Use an outer merge to include all contributors who appear in either index calculation
df_combined_indices = pd.merge(df_em, df_dm,
                               how='outer',
                               on='github_login',
                               suffixes=('_em', '_dm'))

# Handle cases where a contributor might not have activity for one of the indices
# (e.g., no PRs opened/comments for EM, or no code changes/PRs merged for DM)
df_combined_indices['EM'] = df_combined_indices['EM'].fillna(0)
df_combined_indices['DM'] = df_combined_indices['DM'].fillna(0)

# Resolve potential duplicate 'name' columns if a simple merge was used.
# We prefer the 'name' from the EM calculation, or the DM if the EM name is null/NA
df_combined_indices['name'] = df_combined_indices['name_em'].fillna(df_combined_indices['name_dm'])

# Drop the redundant name columns
df_combined_indices = df_combined_indices.drop(columns=['name_em', 'name_dm'], errors='ignore')

# Filter out contributors with zero in both indices for a cleaner plot, unless they are all zeros
# Check if there's any non-zero value at all to decide on filtering
if (df_combined_indices['EM'].sum() > 0) or (df_combined_indices['DM'].sum() > 0):
    df_combined_indices_plot = df_combined_indices[
        (df_combined_indices['EM'] > 0) |
        (df_combined_indices['DM'] > 0)
    ].copy()
else:
    df_combined_indices_plot = df_combined_indices.copy() # Keep all if all are zero


# Create the scatter plot
fig = px.scatter(
    df_combined_indices_plot,
    x='DM', # DM on X-axis
    y='EM',                   # EM on Y-axis
    text=None,       # Removed github_login as text on points
    hover_name='name',         # Show full name on hover
    hover_data={
        'github_login': True, # Keep github_login in hover
        'name': False,       # Don't duplicate hover_name
        'EM': ':.2f',       # Format EM in hover
        'DM': ':.2f' # Format DM in hover
    },
    title='Contribution chart (EM vs DM)',
    labels={
        'EM': 'Engagement Magnitude (EM)',
        'DM': 'Delivery Magnitude (DM)'
    },
    # Removed 'size' from px.scatter to control it via update_traces
    color='github_login', # Assign a unique color to each contributor
    symbol='github_login' # Assign a unique symbol to each contributor
)

# Set a fixed, slightly larger size for all markers directly using update_traces
fig.update_traces(marker=dict(size=10)) # Adjust 'size' to your desired pixel value

# Removed textposition update as text labels are no longer desired
# fig.update_traces(textposition='top center')

# Add lines for quadrants
fig.add_hline(y=0.5, line_dash='dash', line_color='gray', annotation_text="Average EM")
fig.add_vline(x=0.5, line_dash='dash', line_color='gray', annotation_text="Average DM")

# Update layout for better aesthetics and legend position
fig.update_layout(
    height=900, # Increased height to accommodate the legend below
    width=900,
    xaxis=dict(range=[-0.05, 1.05]), # Extend range slightly beyond 0-1
    yaxis=dict(range=[-0.05, 1.05]),
    showlegend=True, # Ensure legend is shown
    legend=dict(
        orientation='v', # Vertical legend
        yanchor='top',   # Anchor to the top of the legend container
        y=1,             # Position at the top right (y=1) of the plot area
        xanchor='left',
        x=1.02           # Position slightly outside the plot area to the right
    )
)

fig.show()
save_fig(fig, 'engagement_delivery_map.html')


Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/engagement_delivery_map.html


* Add REMI e RDMI to dataframe

In [38]:
import pandas as pd

# Select only the necessary columns from df_combined_indices
df_indices_to_merge = df_combined_indices[['github_login', 'EM', 'DM']]

# Ensure there are no existing BCI or Code_Delivery_Index columns in df_people
# that would cause suffixing issues during merge. This is a more robust way to clean up.
cols_to_remove = [col for col in df_people.columns if col.startswith('BCI') or col.startswith('Code_Delivery_Index') or col.startswith('EM') or col.startswith('DM')]
if cols_to_remove:
    df_people = df_people.drop(columns=cols_to_remove)

# Merge with df_people to add EM and DM cleanly
df_people = pd.merge(df_people, df_indices_to_merge,
                     on='github_login',
                     how='left') # Use left merge to keep all rows from df_people

# Fill NaN values for EM and DM with 0 for contributors who had no activity
# These lines should now succeed because the merge would have added the columns.
df_people['EM'] = df_people['EM'].fillna(0.0)
df_people['DM'] = df_people['DM'].fillna(0.0)

print("df_people DataFrame com EM e DM adicionados:")
display(df_people.head())


df_people DataFrame com EM e DM adicionados:


,github_login,name,access,issues_opened_count,issues_reopened_count,issue_comment_event_count,pull_requests_opened_count,pull_requests_merged_count,pull_requests_closed_count,total_additions,total_deletions,scaled_additions,scaled_deletions,EM,DM
0,monacofj,Monaco F. J.,yes,1,0,0,0,0,0,0,0,0.00,0.000,0.078567,0.000000
1,LiviaLelis,Lívia Lelis,yes,0,0,0,0,1,1,1832,250,4.58,0.625,0.000000,0.539475
2,trqt,Guilherme Torquato,yes,2,0,4,1,1,1,64,6,0.16,0.015,0.589256,0.353819
3,jotaefepinho,João Francisco Pinho,yes,0,0,2,0,0,0,0,0,0.00,0.000,0.157135,0.000000
4,chonkachu,N/A,yes,0,0,0,0,0,0,0,0,0.00,0.000,0.000000,0.000000


## Engagement-Delivery distribution


### Engagement (EM) and Delivery (DM) stacked

In [39]:
import plotly.express as px
import pandas as pd

# Select relevant columns from df_people
df_plot_indices = df_people[['github_login', 'name', 'EM', 'DM']].copy()

# Filter out developers who have zero for both EM and DM
df_plot_indices_filtered = df_plot_indices[
    (df_plot_indices['EM'] > 0) | (df_plot_indices['DM'] > 0)
].copy()

if df_plot_indices_filtered.empty:
    print("No contributors with non-zero EM or DM scores found to plot.")
else:
    # Calculate total score for sorting
    df_plot_indices_filtered['total_score'] = df_plot_indices_filtered['EM'] + df_plot_indices_filtered['DM']

    # Sort df_plot_indices_filtered by total_score in descending order
    df_plot_indices_filtered = df_plot_indices_filtered.sort_values(by='total_score', ascending=False)

    # Melt the DataFrame to a long format suitable for stacked bar charts
    df_melted_indices = df_plot_indices_filtered.melt(
        id_vars=['github_login', 'name', 'total_score'], # Include total_score for potential hover or sorting
        value_vars=['EM', 'DM'],
        var_name='Metric',
        value_name='Value'
    )

    # Rename metrics for better readability in the plot
    df_melted_indices['Metric'] = df_melted_indices['Metric'].replace({
        'EM': 'EM (Engagement)',
        'DM': 'DM (Delivery)'
    })

    # Custom order for metrics so EM is 'em baixo' and DM is 'acima'
    metric_order = ['EM (Engagement)', 'DM (Delivery)']
    df_melted_indices['Metric'] = pd.Categorical(df_melted_indices['Metric'], categories=metric_order, ordered=True)

    # Get the sorted list of github_login for xaxis_categoryarray based on total_score
    sorted_github_logins = df_plot_indices_filtered['github_login'].tolist()

    # Create the stacked bar chart using Plotly Express
    fig = px.bar(
        df_melted_indices,
        x='github_login',
        y='Value',
        color='Metric',
        barmode='stack', # This creates stacked bars for each github_login
        title='EM and DM Scores by Contributor (Stacked)',
        labels={
            'github_login': 'Contributor',
            'Value': 'Score',
            'Metric': 'Index Type'
        },
        hover_name='name',
        hover_data={
            'github_login': True, # Keep github_login in hover for context
            'name': False,       # Don't duplicate hover_name
            'Metric': True,
            'Value': ':.2f'      # Format Value to 2 decimal places in hover
        },
        text_auto='.2f' # Display text values automatically
    )

    # Refine plot aesthetics
    fig.update_layout(
        xaxis_title='Contributor',
        yaxis_title='Score',
        legend_title='Index Type',
        xaxis_tickangle=90, # Rotate x-axis labels for readability
        bargap=0.1, # Gap between groups of bars (not very relevant for stacked, but good practice)
        height=600, # Adjust plot height
        xaxis_categoryorder='array', # Preserve the order from the DataFrame
        xaxis_categoryarray=sorted_github_logins
    )

    fig.update_yaxes(rangemode='tozero') # Ensure y-axis starts at zero

    # Set text color to white for all traces and position text inside
    fig.update_traces(textfont_color='white', textposition='inside')

    fig.show()

    save_fig(fig, 'contribution_em_dm_plot.html')


Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/contribution_em_dm_plot.html


### Contribution score

In [40]:
import pandas as pd

# 1. Calculate 'activity': A_factor * (EM>0) + (1-A_factor) * (DM>0)
# Convert boolean to integer (True=1, False=0) for multiplication
df_people['activity'] = (df_people['EM'] > 0).astype(int) * A_factor + \
                        (df_people['DM'] > 0).astype(int) * (1 - A_factor)

# 2. Calculate 'reward' using the formula: R_factor * EM + (1-R_factor) * DM
df_people['reward'] = (R_factor * df_people['EM']) + ((1 - R_factor) * df_people['DM'])

# 3. Calculate the 'score' column as S_factor * activity + (1 - S_factor) * reward (range 0-1)
df_people['score'] = S_factor * df_people['activity'] + (1 - S_factor) * df_people['reward']

# Display the head of the df_people DataFrame to show the newly added 'activity', 'reward', and 'score' columns.
display(df_people.head())


,github_login,name,access,issues_opened_count,issues_reopened_count,issue_comment_event_count,pull_requests_opened_count,pull_requests_merged_count,pull_requests_closed_count,total_additions,total_deletions,scaled_additions,scaled_deletions,EM,DM,activity,reward,score
0,monacofj,Monaco F. J.,yes,1,0,0,0,0,0,0,0,0.00,0.000,0.078567,0.000000,0.5,0.039284,0.269642
1,LiviaLelis,Lívia Lelis,yes,0,0,0,0,1,1,1832,250,4.58,0.625,0.000000,0.539475,0.5,0.269738,0.384869
2,trqt,Guilherme Torquato,yes,2,0,4,1,1,1,64,6,0.16,0.015,0.589256,0.353819,1.0,0.471537,0.735769
3,jotaefepinho,João Francisco Pinho,yes,0,0,2,0,0,0,0,0,0.00,0.000,0.157135,0.000000,0.5,0.078567,0.289284
4,chonkachu,N/A,yes,0,0,0,0,0,0,0,0,0.00,0.000,0.000000,0.000000,0.0,0.000000,0.000000


### Contribution score (CS) plot

In [41]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

# Ensure 'activity', 'reward', and 'score' columns are already calculated in df_people

# Calculate the four individual components of the score
df_people['score_component_activity_em_present'] = S_factor * A_factor * (df_people['EM'] > 0).astype(int)
df_people['score_component_activity_dm_present'] = S_factor * (1 - A_factor) * (df_people['DM'] > 0).astype(int)
df_people['score_component_reward_em_magnitude'] = (1 - S_factor) * R_factor * df_people['EM']
df_people['score_component_reward_dm_magnitude'] = (1 - S_factor) * (1 - R_factor) * df_people['DM']

# 1. Create a new DataFrame, df_contribution_plot, using the score components
df_contribution_plot = df_people[[
    'github_login', 'name', 'score',
    'score_component_activity_em_present',
    'score_component_activity_dm_present',
    'score_component_reward_em_magnitude',
    'score_component_reward_dm_magnitude'
]].copy()

# 2. Filter df_contribution_plot to include only contributors where score > 0
df_contribution_plot = df_contribution_plot[df_contribution_plot['score'] > 0].copy()

# 3. Melt df_contribution_plot into a long format suitable for stacking
df_melted_score = df_contribution_plot.melt(
    id_vars=['github_login', 'name', 'score'],
    value_vars=[
        'score_component_activity_em_present',
        'score_component_activity_dm_present',
        'score_component_reward_em_magnitude',
        'score_component_reward_dm_magnitude'
    ],
    var_name='Component Type',
    value_name='Value'
)

# 4. Rename component types for better plot labels
df_melted_score['Component Type'] = df_melted_score['Component Type'].replace({
    'score_component_activity_em_present': 'Activity (EM Present)',
    'score_component_activity_dm_present': 'Activity (DM Present)',
    'score_component_reward_em_magnitude': 'Reward (EM Magnitude)',
    'score_component_reward_dm_magnitude': 'Reward (DM Magnitude)'
})

# 5. Sort the melted DataFrame by total score in descending order
df_melted_score = df_melted_score.sort_values(by='score', ascending=False)

# Get the sorted list of github_login for xaxis_categoryarray
sorted_github_logins_for_score = df_melted_score['github_login'].unique().tolist()

# Define custom colors for the components
# Two shades of blue for activity components, two shades of green for reward components
component_colors = {
    'Activity (EM Present)': '#2A52BE',  # Darker Blue
    'Activity (DM Present)': '#6495ED',  # Slightly darker CornflowerBlue
    'Reward (EM Magnitude)': '#228B22',  # Darker Green (ForestGreen)
    'Reward (DM Magnitude)': '#66CDAA'   # Slightly darker MediumAquamarine
}

# Define component order for consistent stacking
component_order = [
    'Activity (EM Present)',
    'Activity (DM Present)',
    'Reward (EM Magnitude)',
    'Reward (DM Magnitude)'
]
df_melted_score['Component Type'] = pd.Categorical(df_melted_score['Component Type'], categories=component_order, ordered=True)

# 6. Create a stacked bar chart using Plotly Express
fig = px.bar(
    df_melted_score,
    x='github_login',
    y='Value',
    color='Component Type',
    color_discrete_map=component_colors, # Apply custom colors
    barmode='stack',
    title='Contributor Score (CS)',
    labels={
        'github_login': 'Contributor',
        'Value': 'Score Component Value',
        'Component Type': 'Score Component'
    },
    hover_name='name',
    hover_data={
        'github_login': True,
        'name': False,
        'Component Type': True,
        'Value': ':.1f', # Format component value to 1 decimal place
        'score': ':.1f' # Display total score in hover to 1 decimal place
    },
    text_auto='.1f', # Display component values on top of bars, formatted to 1 decimal place
    category_orders={'Component Type': component_order} # Explicitly set stacking order for color variable
)

# Add total score as text on top of each bar
fig.add_trace(go.Scatter(
    x=df_contribution_plot['github_login'],
    y=df_contribution_plot['score'],
    mode='text',
    text=[f'{s:.1f}' for s in df_contribution_plot['score']], # Format total score to 1 decimal place
    textposition='top center',
    textfont=dict(color='black', size=10),
    showlegend=False,
    hoverinfo='none'
))

# f. Adjust the x-axis to show labels vertically and preserve the order from the sorted DataFrame
fig.update_xaxes(tickangle=-90, tickfont=dict(size=10),
                 categoryorder='array',
                 categoryarray=sorted_github_logins_for_score)

# g. Ensure the y-axis starts at zero (`rangemode='tozero'`) and has a maximum range slightly above 1
fig.update_yaxes(rangemode='tozero', title_text='Score', range=[0, 1.1]) # Adjusted range to 0-1.1

# h. Display the Value on top of the bars in white text, positioned inside (for components)
fig.update_traces(textfont_color='white', textposition='inside', selector=dict(type='bar'))

# Add horizontal dashed line at 0.5
fig.add_hline(y=0.5, line_dash='dash', line_color='black', line_width=1.5)

# Display the plot
fig.show()

print("Overall Score calculation and stacked bar chart generation complete.")
save_fig(fig, 'score_plot.html')


Overall Score calculation and stacked bar chart generation complete.
Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/score_plot.html


### Score percentiles

In [42]:
import plotly.express as px
import pandas as pd

# Filter out contributors with a score of 0 for a more meaningful quartile analysis
df_active_scores = df_people[df_people['score'] > 0].copy()

if df_active_scores.empty:
    print("No contributors with a score > 0 found to plot quartiles.")
else:
    # Create a box plot for the 'score'
    fig = px.box(
        df_active_scores,
        y='score',
        title='Quartile Distribution of Contribution Scores',
        labels={
            'score': 'Contributor Score'
        },
        hover_data={'github_login': True, 'name': True, 'score': ':.2f'}
    )

    # Customize layout for better readability
    fig.update_layout(
        yaxis_title='Score',
        yaxis_range=[0, 1], # Ensure y-axis covers the full score range from 0 to 1
        yaxis=dict(dtick=0.1), # Set grid lines every 0.1 units
        boxmode='overlay' # Overlay points if multiple boxes (not applicable for single score column)
    )

    # Add individual data points as a scatter plot overlay (optional, for detail)
    fig.add_trace(px.scatter(
        df_active_scores,
        y='score',
        hover_name='name',
        hover_data={'github_login': True, 'name': True, 'score': ':.2f'}
    ).data[0])

    fig.show()
    save_fig(fig, 'score_percentiles_box_plot.html')

    print("Box plot for contributor scores generated.")

    # Print quartile values with explanations
    q1 = df_active_scores['score'].quantile(0.25)
    median = df_active_scores['score'].quantile(0.50)
    q3 = df_active_scores['score'].quantile(0.75)
    max_score = df_active_scores['score'].max()
    min_score = df_active_scores['score'].min()

    print("\n--- Contributor Score Quartiles ---")
    print(f"* The lowest active score is: {min_score:.2f}")
    print(f"* 25% of contributors have a score up to: {q1:.2f}")
    print(f"* 50% of contributors (the median) have a score up to: {median:.2f}")
    print(f"* 75% of contributors have a score up to: {q3:.2f}")
    print(f"* The highest active score is: {max_score:.2f}")

Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/score_percentiles_box_plot.html
Box plot for contributor scores generated.

--- Contributor Score Quartiles ---
* The lowest active score is: 0.25
* 25% of contributors have a score up to: 0.27
* 50% of contributors (the median) have a score up to: 0.37
* 75% of contributors have a score up to: 0.62
* The highest active score is: 0.89


### Score histogram

In [43]:
import plotly.express as px
import pandas as pd

# Filter out contributors with a score of 0 for a more meaningful histogram
df_active_scores_hist = df_people[df_people['score'] > 0].copy()

if df_active_scores_hist.empty:
    print("No contributors with a score > 0 found to plot a histogram.")
else:
    # Create a histogram for the 'score'
    fig = px.histogram(
        df_active_scores_hist,
        x='score',
        nbins=20, # Adjust number of bins as needed
        title='Distribution of Contribution Scores',
        labels={
            'score': 'Contributor Score',
            'count': 'Number of Contributors'
        },
        hover_data={'name': True, 'score': ':.2f'}
    )

    # Customize layout for better readability
    fig.update_layout(
        xaxis_title='Contribution Score',
        yaxis_title='Number of Contributors',
        xaxis_range=[0, 1], # Ensure x-axis covers the full score range
        bargap=0.1 # Gap between bars
    )

    # Calculate mean and median
    mean_score = df_active_scores_hist['score'].mean()
    median_score = df_active_scores_hist['score'].median()

    # Add mean line
    fig.add_vline(x=mean_score, line_width=2, line_dash="dash", line_color="red",
                  annotation_text=f"Mean: {mean_score:.2f}",
                  annotation_position="top right")

    # Add median line
    fig.add_vline(x=median_score, line_width=2, line_dash="dot", line_color="green",
                  annotation_text=f"Median: {median_score:.2f}",
                  annotation_position="top left")

    fig.show()

    print("Histogram for contributor scores generated.")
    save_fig(fig, 'score_histogram.html')

Histogram for contributor scores generated.
Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/score_histogram.html


### Score proportions

In [44]:
# cellname : scopre-pizza
import plotly.express as px
import pandas as pd

# 1. Categorize the 'score' column with new names
def categorize_score(score):
    if score == 0:
        return 'Zero (score = 0)'
    elif 0 < score < 0.5:
        return 'Low (0 < score < 0.5)'
    else:
        return 'Good (0.5 <= score)'

df_people['score_category'] = df_people['score'].apply(categorize_score)

# 2. Count the occurrences in each category
score_counts = df_people['score_category'].value_counts().reset_index()
score_counts.columns = ['Category', 'Count']

# 3. Define custom colors for the categories
category_colors = {
    'Zero (score = 0)': '#aaaa80',       # Lighter gray
    'Low (0 < score < 0.5)': 'darkred',    # Darker red
    'Good (0.5 <= score)': 'darkcyan'            # Cyan
}

# Ensure category order for consistent plotting
category_order = ['Good (0.5 <= score)', 'Low (0 < score < 0.5)', 'Zero (score = 0)']
score_counts['Category'] = pd.Categorical(score_counts['Category'], categories=category_order, ordered=True)
score_counts = score_counts.sort_values('Category')

# 5. Create the pie chart
fig = px.pie(
    score_counts,
    values='Count',
    names='Category', # Use the raw category name for slice identification
    title='Distribuição de Contribuidores por Faixa de Score',
    color='Category',
    color_discrete_map=category_colors,
    hover_data=['Count', 'Category']
)

# Update traces to show percentage and formatted name as text, outside
fig.update_traces(textposition='outside', textinfo='percent', texttemplate="<b>%{label}</b><br>%{percent:.1%}<br>(%{value})", textfont_color='black')

fig.show()
save_fig(fig, 'score_proportions_pie_chart.html')

Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/score_proportions_pie_chart.html


### Aggregate indices

In [45]:
import pandas as pd
import plotly.express as px
import numpy as np

# Prepare the data for quartile analysis
# Use the original activity and reward values, which are already normalized 0-1
df_people_quartiles = df_people.copy()

# Select the relevant columns for the box plots
metrics_for_quartiles = df_people_quartiles[['github_login', 'EM', 'DM', 'activity', 'reward']]

# Filter out contributors who have 0 across all these metrics for meaningful quartiles
df_active_metrics = metrics_for_quartiles[
    (metrics_for_quartiles['EM'] > 0) |
    (metrics_for_quartiles['DM'] > 0) |
    (metrics_for_quartiles['activity'] > 0) |
    (metrics_for_quartiles['reward'] > 0)
].copy()

if df_active_metrics.empty:
    print("No active contributors found to display quartile distributions.")
else:
    # Melt the DataFrame to a long format suitable for px.box for side-by-side plots
    df_melted_metrics = df_active_metrics.melt(
        id_vars=['github_login'],
        value_vars=['EM', 'DM', 'activity', 'reward'],
        var_name='Metric',
        value_name='Value'
    )

    # Rename metrics for better display in the plot
    df_melted_metrics['Metric'] = df_melted_metrics['Metric'].replace({
        'EM': 'EM (Engagement)',
        'DM': 'DM (Delivery)',
        'activity': 'Activity',
        'reward': 'Reward'
    })

    # Define explicit order for metrics for consistent plotting
    metric_order = ['EM (Engagement)', 'DM (Delivery)', 'Activity', 'Reward']
    df_melted_metrics['Metric'] = pd.Categorical(df_melted_metrics['Metric'], categories=metric_order, ordered=True)

    # Create the box plot
    fig = px.box(df_melted_metrics,
                 x='Metric',
                 y='Value',
                 color='Metric', # Color boxes by metric type
                 title='Quartile distribution of tide metrics',
                 labels={'Metric': 'Metric', 'Value': 'Value'}, # Updated labels to English
                 hover_data={'github_login': True, 'Value': ':.2f'},
                 points='all' # Display all individual points
                )

    # Customize layout
    fig.update_layout(
        xaxis_title='Metric',
        yaxis_title='Value',
        xaxis_tickangle=-45,
        height=600,
        showlegend=False, # Legend not needed as colors are explained by x-axis
        yaxis_range=[0, 1], # Set y-axis range to 0-1
        yaxis=dict(dtick=0.1)
    )

    fig.show()
    save_fig(fig, 'metric_quartiles_box_plot.html')

    print("\n--- Quartile Values for Each Metric ---")
    for metric in ['EM', 'DM', 'activity', 'reward']:
        series = df_active_metrics[metric]
        if not series.empty:
            q1 = series.quantile(0.25)
            median = series.quantile(0.50)
            q3 = series.quantile(0.75)
            max_val = series.max()
            min_val = series.min()
            mean_val = series.mean()

            print(f"\nMetric: {metric.replace('EM', 'Engagement Magnitude').replace('DM', 'Delivery Magnitude')}")
            print(f"  Mean: {mean_val:.2f}")
            print(f"  Min: {min_val:.2f}")
            print(f"  Q1 (25th percentile): {q1:.2f}")
            print(f"  Median (50th percentile): {median:.2f}")
            print(f"  Q3 (75th percentile): {q3:.2f}")
            print(f"  Max: {max_val:.2f}")
        else:
            print(f"\nMetric: {metric.replace('EM', 'Engagement Magnitude').replace('DM', 'Delivery Magnitude')} - No active data.")


Figure saved to: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/metric_quartiles_box_plot.html

--- Quartile Values for Each Metric ---

Metric: Engagement Magnitude
  Mean: 0.30
  Min: 0.00
  Q1 (25th percentile): 0.08
  Median (50th percentile): 0.20
  Q3 (75th percentile): 0.47
  Max: 1.00

Metric: Delivery Magnitude
  Mean: 0.19
  Min: 0.00
  Q1 (25th percentile): 0.00
  Median (50th percentile): 0.00
  Q3 (75th percentile): 0.35
  Max: 0.84

Metric: activity
  Mean: 0.69
  Min: 0.50
  Q1 (25th percentile): 0.50
  Median (50th percentile): 0.50
  Q3 (75th percentile): 1.00
  Max: 1.00

Metric: reward
  Mean: 0.24
  Min: 0.00
  Q1 (25th percentile): 0.05
  Median (50th percentile): 0.18
  Q3 (75th percentile): 0.33
  Max: 0.77


# SAVE DATAFRAME

In [46]:
import os

# Ensure output_folder and df_people are defined from previous cells

output_csv_path = os.path.join(output_folder, 'developers_data.csv')
df_people.to_csv(output_csv_path, index=False)

print(f"DataFrame 'df_people' successfully saved to {output_csv_path}")

DataFrame 'df_people' successfully saved to /content/gdrive/My Drive/naja-2025-11-26_22-47-07/developers_data.csv


# CREATE REPORT

## Section descriptions

In [47]:
# --- Report Description Variables ---

# Project Summary
summary_description = "This section provides a high-level overview of the GitHub repository, including its basic metadata and general statistics."

# Project Overview
overview_description = "This section summarizes the key demographic data for contributors and collaborators within the analyzed period."

# Inactive Developers
inactive_devs_description = "This section lists developers who are either collaborators or external contributors but did not record any activity (issues opened/reopened, comments, PRs opened/merged, code changes) within the selected analysis period."
inactive_devs_found_text = "Developer(s) with No Contributions in the Period:"
inactive_devs_none_text = "All developers have made at least one contribution in the selected period!"

# All Primitive Events
primitive_events_description = "This chart displays the raw count of various GitHub events (issues opened, issues reopened, comments, PRs opened, PRs merged, lines added, lines removed) per contributor. It provides a granular view of individual developer activity."

# All Aggregate Events
aggregate_events_description = "This chart aggregates primitive events into broader categories: 'Issues Raised' (opened + reopened), 'Comments' (on issues/PRs), 'PRs Opened', 'PRs Merged', and 'Code Changes' (additions + deletions). It offers a summarized view of contribution types."

# WB
wbi_description = "The WB (Workload Balance), calculated as (PRs Closed - PRs Opened) / (PRs Closed + PRs Opened), measures the balance between new work introduced (PRs opened) and work completed (PRs closed) in the project. A positive value indicates clearing the backlog, while a negative value suggests accumulation."

# RB
rbi_description = "The RB (Resolution Balance), calculated as (PRs Submitted - Issues Raised) / (PRs Submitted + Issues Raised), reflects the balance between problem identification (issues raised) and solution implementation (PRs submitted). A positive value implies a focus on implementation, while a negative value suggests more issues are being identified than solutions proposed."

# Project Performance Map
performance_map_description = "This map plots the Workload Balance (WB) against the Resolution Balance (RB) on a polar coordinate system. It visually represents the overall project dynamics, indicating whether the team is accumulating/clearing work and focusing on planning/implementing tasks."

# PRs Close Time section
pr_close_time_description = "This section analyzes the lifecycle of Pull Requests, specifically focusing on the time taken to close them. It categorizes PRs based on their creation and closure dates relative to the analysis period."

# Developers Overview - Engagement
engagement_plot_description = "This plot visualizes individual contributor engagement by mapping 'PRs Opened' against 'Discussion' (Issues Opened + Comments). It helps identify developers who are more conversational versus those focused on initiating code contributions."
remi_index_description = "EM quantifies a developer's overall engagement by taking the Euclidean norm of their normalized 'Discussion' and 'PRs Opened' scores. It intuitively measures the intensity of a developer's participatory and initiatory actions, ranging from 0 (no engagement) to 1 (maximum engagement)."

# Developers Overview - Delivery
delivery_plot_description = "This plot visualizes individual contributor delivery by mapping 'Code Changes' (lines added + deleted) against 'PRs Merged'. It helps identify developers who are generating significant code volume versus those focused on successfully integrating code into the main branch."
rdmi_index_description = "DM quantifies a developer's overall delivery by taking the Euclidean norm of their normalized 'Code Changes' and 'PRs Merged' scores. It intuitively measures the overall strength of a developer's effective code delivery, ranging from 0 (no delivery) to 1 (maximum delivery)."

# Engagement-Delivery Map
engagement_delivery_map_description = "This map plots each developer's EM against their DM score. It categorizes developers based on their relative engagement and delivery levels, helping identify different contributor profiles (e.g., highly engaged/low delivery vs. low engagement/high delivery)."

# Contribution (EM+DM) Plot
contribution_plot_description = "This stacked bar chart displays each developer's EM and DM scores side-by-side. It offers a combined view of individual engagement and delivery magnitudes, allowing for quick comparison across contributors."

# Score Plot
score_plot_description = "The overall score for each contributor is calculated as a weighted sum of 'Activity' and 'Reward', using the formula `S * Activity + (1 - S) * Reward`. This provides a comprehensive measure between 0 and 1. 'Activity' measures minimal participation, while 'Reward' measures participation intensity based on EM and DM. This chart displays the stacked components of 'Activity' and 'Reward' contributing to the total score."
score_percentiles_description = "This box plot visualizes the distribution of individual contributor scores, showing quartiles (25th, 50th, 75th percentiles), minimum, and maximum values. It helps in understanding the spread and central tendency of scores across active developers."
score_histogram_description = "This histogram shows the frequency distribution of contributor scores, indicating how many developers fall within specific score ranges. It helps identify patterns in scoring, such as concentration around certain values or presence of multiple peaks."
score_proportions_description = "This pie chart categorizes contributors into three score ranges: 'Zero' (score = 0), 'Low' (0 < score < 0.5), and 'Good' (0.5 <= score). It illustrates the proportion of developers in each category, providing a high-level overview of the team's overall performance distribution."

# Aggregate Indices
metric_quartiles_description = "This box plot displays the quartile distribution (25th, 50th, 75th percentiles) of individual EM, DM, Activity, and Reward scores across all active contributors. It provides insights into the spread and central tendency of these key metrics among developers."


## Report structure

In [48]:
import os
from datetime import datetime

# Ensure output_folder is defined from previous cells
# output_folder = '/content/gdrive/My Drive/devstats_report' # Uncomment if running this cell independently

html_content_parts = []

# Ensure analysis_run_date is available. It should be defined in a preceding cell.
# Add a safety check for analysis_run_date
if 'analysis_run_date' not in globals():
    analysis_run_date = "Analysis date not available - please run the 'Get repository info' cell." # Fallback
    print("Warning: 'analysis_run_date' not found. Using fallback message.")

print(f"Using analysis_run_date for report: {analysis_run_date}")

# 1. HTML Head and Styles
html_content_parts.append(f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>TideStats Repository Report</title>
    <style>
        body {{
            font-family: 'Arial', sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #f4f7f6;
            color: #333;
            line-height: 1.6;
        }}
        .container {{
            max-width: 1200px;
            margin: 20px auto;
            background-color: #ffffff;
            padding: 30px;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        }}
        h1, h2, h3, h4, h5, h6 {{
            color: #2c3e50;
        }}
        h1 {{
            text-align: center;
            color: #2196f3;
            margin-bottom: 30px;
        }}
        .section {{
            margin-bottom: 40px;
            border-bottom: 1px solid #eee;
            padding-bottom: 20px;
        }}
        .section:last-child {{
            border-bottom: none;
        }}
        .figure-container {{
            max-width: 1000px; /* Increased max-width for centering wider iframes */
            margin: 20px auto; /* Center the figure container itself */
            text-align: center; /* Center inline content like h3 */
            border: 1px solid #ddd;
            padding: 15px;
            border-radius: 5px;
            background-color: #f9f9f9;
        }}
        .figure-container h3 {{
            margin-top: 0;
            color: #555;
        }}
        .figure-container iframe {{
            /* Removed default width: 100%; to allow individual width or auto-sizing */
            height: 600px; /* Re-added default height for plotly figures to prevent flattening */
            width: 100%; /* Allow iframe to fill its container */
            border: none;
            display: block; /* Treat iframe as a block element */
            margin: 0 auto; /* Center iframe horizontally */
            text-align: center; /* Center iframe text */
        }}

        .subsection {{
            margin-bottom: 20px;
            border-bottom: 1px solid #eee;
            padding-bottom: 15px;
        }}
    </style>
</head>
<body>
    <div class="container">
        <h1>TideStats Repository Report</h1>
"""
)

# 2. Project Summary
html_content_parts.append(f"""
        <div class="section">
            <h2>Project Summary</h2>
            <p>{summary_description}</p>
            <p><b>Analysis Run Date:</b> {analysis_run_date}</p>
            <p><b>Repository Name:</b> {repo.name}</p>
            <p><b>Description:</b> {repo.description if repo.description else 'No description provided.'}</p>
            <p><b>URL:</b> <a href="{repo.html_url}">{repo.html_url}</a></p>
            <p><b>Stars:</b> {repo.stargazers_count}</p>
            <p><b>Forks:</b> {repo.forks_count}</p>
            <p><b>Created At:</b> {repo.created_at.strftime('%Y-%m-%d %H:%M:%S')}</p>
            <p><b>Last Updated At:</b> {repo.updated_at.strftime('%Y-%m-%d %H:%M:%S')}</p>
        </div>
"""
)

# 3. Project Overview
html_content_parts.append(f"""
        <div class="section">
            <h2>Project Overview</h2>
            <p>{overview_description}</p>
            <p><b>Analysis Period:</b> {selected_period_str}</p>
            <p><b>Number of Collaborators (Core Devs):</b> {num_core_devs}</p>
            <p><b>Number of Contributors without Repo Access (Externals):</b> {num_externals}</p>
        </div>
"""
)

# 4. Inactive Developers
inactive_devs_html = f"""
        <div class="section">
            <h2>Inactive Developers</h2>
            <p>{inactive_devs_description}</p>
"""
if not inactive_devs_df.empty:
    inactive_devs_html += f"""
            <p>{len(inactive_devs_df)} {inactive_devs_found_text}</p>
            <ul>
"""
    for index, row in inactive_devs_df.iterrows():
        dev_name = row['name'] if row['name'] != 'N/A' else 'Name not available'
        inactive_devs_html += f"                <li>{row['github_login']} ({dev_name})</li>\n"
else:
    inactive_devs_html += f"            <p>{inactive_devs_none_text}</p>\n"
inactive_devs_html += "        </div>\n"
html_content_parts.append(inactive_devs_html)

# 5. All Primitive Events
html_content_parts.append(f"""
        <div class="section">
            <h2>All Primitive Events</h2>
            <p>{primitive_events_description}</p>
            <div class="figure-container">
                <h3>Event Count per Contributor</h3>
                <iframe src="./all_primitive_events.html"></iframe>
            </div>
        </div>
"""
)

# 6. All Aggregate Events
html_content_parts.append(f"""
        <div class="section">
            <h2>All Aggregate Events</h2>
            <p>{aggregate_events_description}</p>
            <div class="figure-container">
                <h3>Aggregate Event Count per Contributor</h3>
                <iframe src="./all_aggregate_events.html"></iframe>
            </div>
        </div>
"""
)

# 7. WB
html_content_parts.append(f"""
        <div class="section">
            <h2>Workload Balance (WB)</h2>
            <p>{wbi_description}</p>
            <div class="figure-container">
                <h3>Workload Balance (WB)</h3>
                <iframe src="./workload_balance.html" style="height: 200px; width: 880px;"></iframe>
            </div>
        </div>
"""
)

# 8. RB
html_content_parts.append(f"""
        <div class="section">
            <h2>Resolution Balance (RB)</h2>
            <p>{rbi_description}</p>
            <div class="figure-container">
                <h3>Resolution Balance (RB)</h3>
                <iframe src="./resolution_balance.html" style="height: 200px; width: 880px;"></iframe>
            </div>
        </div>
"""
)

# 9. Project Performance Map
html_content_parts.append(f"""
        <div class="section">
            <h2>Project Performance Map (WB x RB)</h2>
            <p>{performance_map_description}</p>
            <div class="figure-container">
                <h3>Project Performance Map</h3>
                <iframe src="./wb_rb_map.html" style="height: 700px; width: 960px;"></iframe>
            </div>
        </div>
"""
)

# 10. PRs Close Time section
html_content_parts.append(f"""
        <div class="section">
            <h2>PRs Close Time</h2>
            <p>{pr_close_time_description}</p>
            <div class="figure-container">
                <h3>Distribution of PR Close Times</h3>
                <iframe src="./pr_close_times_histogram.html"></iframe>
            </div>
            <div class="figure-container">
                <h3>PRs Opened and Closed in Period vs. Time to Close with Quartiles</h3>
                <iframe src="./pr_close_time_scatter_plot.html" style="height: 700px;"></iframe>
            </div>
            <div class="figure-container">
                <h3>Distribution of Pull Request Status in the Period</h3>
                <iframe src="./pr_categories_pie_chart.html"></iframe>
            </div>
        </div>
"""
)

# 11. Developers Overview - Engagement
html_content_parts.append(f"""
        <div class="section">
            <h2>Developers Overview</h2>
            <div class="subsection">
                <h3>Engagement Plot</h3>
                <p>{engagement_plot_description}</p>
                <div class="figure-container">
                    <iframe src="./engagement_plot.html" style="height: 700px;"></iframe>
                </div>
            </div>
            <div class="subsection">
                <h3>Engagement Magnitude Index (EM)</h3>
                <p>{remi_index_description}</p>
                <div class="figure-container">
                    <iframe src="./em_index.html"></iframe>
                </div>
            </div>
"""
)

# 12. Developers Overview - Delivery
html_content_parts.append(f"""
            <div class="subsection">
                <h3>Delivery Plot</h3>
                <p>{delivery_plot_description}</p>
                <div class="figure-container">
                    <iframe src="./delivery_plot.html" style="height: 900px;"></iframe>
                </div>
            </div>
            <div class="subsection">
                <h3>Delivery Magnitude Index (DM)</h3>
                <p>{rdmi_index_description}</p>
                <div class="figure-container">
                    <iframe src="./dm_index.html"></iframe>
                </div>
            </div>
"""
)

# 13. Engagement-Delivery Map
html_content_parts.append(f"""
            <div class="subsection">
                <h3>Engagement-Delivery Map</h3>
                <p>{engagement_delivery_map_description}</p>
                <div class="figure-container">
                    <iframe src="./engagement_delivery_map.html" style="height: 1000px;"></iframe>
                </div>
            </div>
"""
)

# 14. Contribution (EM+DM) Plot
html_content_parts.append(f"""
            <div class="subsection">
                <h3>Contribution (EM+DM) Plot</h3>
                <p>{contribution_plot_description}</p>
                <div class="figure-container">
                    <iframe src="./contribution_em_dm_plot.html" style="height: 700px;"></iframe>
                </div>
            </div>
"""
)

# 15. Score Plot
html_content_parts.append(f"""
            <div class="subsection">
                <h3>Contributor Overall Score</h3>
                <p>{score_plot_description}</p>
                <div class="figure-container">
                    <iframe src="./score_plot.html"></iframe>
                </div>
            </div>
            <div class="subsection">
                <h3>Score Percentiles</h3>
                <p>{score_percentiles_description}</p>
                <div class="figure-container">
                    <iframe src="./score_percentiles_box_plot.html"></iframe>
                </div>
            </div>
            <div class="subsection">
                <h3>Score Histogram</h3>
                <p>{score_histogram_description}</p>
                <div class="figure-container">
                    <iframe src="./score_histogram.html"></iframe>
                </div>
            </div>
            <div class="subsection">
                <h3>Score Proportions</h3>
                <p>{score_proportions_description}</p>
                <div class="figure-container">
                    <iframe src="./score_proportions_pie_chart.html"></iframe>
                </div>
            </div>
        </div> <!-- Close Developers Overview section -->
"""
)

# 16. Aggregate Indices Header
html_content_parts.append(f"""
        <div class="section">
            <h2>Aggregate Indices</h2>
"""
)

# 18. Quartile Distribution of Key Development Metrics
html_content_parts.append(f"""
            <div class="subsection">
                <h3>Quartile Distribution of Key Development Metrics</h3>
                <p>{metric_quartiles_description}</p>
                <div class="figure-container">
                    <iframe src="./metric_quartiles_box_plot.html" style="height: 700px;"></iframe>
                </div>
            </div>
        </div> <!-- Close Aggregate Indices section -->
"""
)

# 19. Final closing tags
html_content_parts.append("""
    </div> <!-- Close container -->
</body>
</html>
"""
)

# Join all parts and write to file
final_html_content = "".join(html_content_parts)

html_file_path = os.path.join(output_folder, 'index.html')

with open(html_file_path, 'w') as f:
    f.write(final_html_content)

print(f"Finalized HTML report created at: {html_file_path}")

Using analysis_run_date for report: 2025-11-26 22:47:36
Finalized HTML report created at: /content/gdrive/My Drive/naja-2025-11-26_22-47-07/index.html
